The goal is to design an algorithm that can evolve the entire structure of the neural network based only on the data .
# first of all import the required packages.
1-random for random numbers. \
2-numpy for array operations. \
3- heapq for sorting. \
4- copy. \
5- os for the path of the files.\
6- math.\
7- statistics. \
8- time.\
9- numba for jit compiler.\
10- tqdm for progress bar.\
11- matplotlib for feedback.

In [2]:
import random
import numpy as np
import heapq
import copy  
import os
import math
import statistics
import time
import numba
from numba.experimental import jitclass 
from tqdm import tqdm
import matplotlib.pyplot as plt

#declaring the algorithm's parameters.
determine things like the number of inputs , number of output , the maximum and the minimum number of neurons in each layer, mutation rate and mutation accuracy and many others .

In [3]:
INPUT = 60 #number of nodes in the input layer 
OUTPUT = 6 #the number of nodes in the output layer
MUTATION_RATE_MAX = 0.15 # maximum allowed mutation rate
MUTATION_RATE = 0.05 # 5% mutation 
MUTATION_RATE_MIN = 0.01 #minimum allowed mutation rate
#since the mutations in the final layers tends to be more influancing on the output 
#the mutation rate at the early layers is increased to active fareness
MUTATION_DELTA = 0.03 # depends on the distance bitween the wight matrex and the output
HIDDEN_1_MAX = 120 #maximum numper of nodes in  hidden_1 layer
HIDDEN_1_MIN = 100 #minimum number of nodes in hidden layer 1
HIDDEN_2_MAX = 120 #maximum numper of nodes in the hidden_2 layer
HIDDEN_2_MIN = 100 #minimum number of nodes in hidden layer 2
HIDDEN_3_MAX = 100 #maximum numper of nodes in the hidden_3 layer
HIDDEN_3_MIN = 80 #minimum number of nodes in hiedden 3 layer 
HIDDEN_4_MAX = 100 #maximum numper of nodes in  hidden_4 layer
HIDDEN_4_MIN = 80 #minimum number of nodes in hiedden 4 layer 
HIDDEN_5_MAX = 80 #maximum numper of nodes in the hidden_5 layer
HIDDEN_5_MIN = 60 #minimum number of nodes in hidden layer 5
HIDDEN_6_MAX = 80 #maximum numper of nodes in the hidden_6 layer
HIDDEN_6_MIN = 60 #minimum number of nodes in hidden layer 6
HIDDEN_7_MAX = 60 #maximum numper of nodes in  hidden_7 layer
HIDDEN_7_MIN = 40 #minimum number of nodes in hidden layer 6
HIDDEN_8_MAX = 60 #maximum numper of nodes in the hidden_8 layer
HIDDEN_8_MIN = 40 #minimum number of nodes in hidden layer 6
HIDDEN_9_MAX = 40 #maximum numper of nodes in the hidden_9 layer
HIDDEN_9_MIN = 30 #minimum number of nodes in hidden layer 6
INITIAL_BMAX = 0.1 #the initial maximum value of baises
INITIAL_BMIN =-0.1 #the initial minimum value of baises
INITIAL_WMAX = 0.1 #the initial maximum value of weightes
INITIAL_WMIN =-0.1 #the initial minimum value of weightes
MAX_NUM_OF_GEN = 500 # maximum number of generations 
NUM_OF_Z_C = 0 #the initial number of zero creatures 
SIZE_OF_POPULATION = 48 # size of the population
MUT_ACC_MAX = 0.5e-1 #maximum mutation steps
MUT_ACC = 1e-3 # initial mutation accurasy 
BEST_FIT = 6 #number of creatures that will servive every generation
RATIO_OF_MUT = 2 #the ratio of mutated creatures to the crossover creatures 
STATE_MAX = 80 # the maximum number of state veriables
STATE_MIN = 60 # the minimum number of states allowed to the creature 
RAND_PER_GEN = 0 #the number of new random creatures for each generation 
SELECTION_CONTROL_MEM = 20 #the memory of selection control
SELECTION_RULE = 0 # the selection function 
ST_DIV  = 0.5 # the standerd deviation used in the fitness function
fitness_mem = [0]*SELECTION_CONTROL_MEM # fitness buffer
accuracy_mem = [0]*SELECTION_CONTROL_MEM # accuracy buffer
trainingInput = np.array([],dtype = np.float32)
trainingOutput = np.array([],dtype = np.float32)
np.warnings.filterwarnings('ignore') # ignore warnings

#fitness and accuracy record.
the valuse of the fitness and accuracy of the best creature according to some policy are recorded . 

In [4]:
fitt_plot = []
acc_plot = []

# determine the path for the data and results to be saved in
in this case the folder **GEN** will contain the training set and the (Architectue/wieghts) files of the generated nueral network, as well as the status file 

In [5]:
path = 'drive/MyDrive/GEN'

# creaure class declaration 
the class contains all the genetic inforation for the creature, and the methods required to make use of these information for more detals please read our thesis .

In [6]:
@jitclass([('fitness' , numba.float32 ),
	('accuracy' , numba.float32),
	('NumOfHid_1' , numba.int32),
	('NumOfHid_2' , numba.int32),
	('NumOfHid_3' , numba.int32),
	('NumOfHid_4' , numba.int32),
	('NumOfHid_5' , numba.int32),
	('NumOfHid_6' , numba.int32),
	('NumOfHid_7' , numba.int32),
	('NumOfHid_8' , numba.int32),
	('NumOfHid_9' , numba.int32),
	('NumOfStates' , numba.int32),
	('BaisIn' , numba.float64[:,:]),
	('BaisOut' , numba.float64[:,:]),
	('BaisHid_1' , numba.float64[:,:]),
	('BaisHid_2' , numba.float64[:,:]),
	('BaisHid_3' , numba.float64[:,:]),
	('BaisHid_4' , numba.float64[:,:]),
	('BaisHid_5' , numba.float64[:,:]),
	('BaisHid_6' , numba.float64[:,:]),
	('BaisHid_7' , numba.float64[:,:]),
	('BaisHid_8' , numba.float64[:,:]),
	('BaisHid_9' , numba.float64[:,:]),
	('wieghts_1' , numba.float64[:,:]),
	('wieghts_2' , numba.float64[:,:]),
	('wieghts_3' , numba.float64[:,:]),
	('wieghts_4' , numba.float64[:,:]),
	('wieghts_5' , numba.float64[:,:]),
	('wieghts_6' , numba.float64[:,:]),
	('wieghts_7' , numba.float64[:,:]),
	('wieghts_8' , numba.float64[:,:]),
	('wieghts_9' , numba.float64[:,:]),
	('wieghts_10' , numba.float64[:,:]),
	('wieghts_11' , numba.float64[:,:]),
	('wieghts_12' , numba.float64[:,:]),
	('wieghts_13' , numba.float64[:,:]),
	('wieghts_14' , numba.float64[:,:]),
	('wieghts_15' , numba.float64[:,:]),
	('wieghts_16' , numba.float64[:,:]),
	('wieghts_17' , numba.float64[:,:]),
	('wieghts_18' , numba.float64[:,:]),
	('wieghts_19' , numba.float64[:,:]),
	('wieghts_20' , numba.float64[:,:]),
	('wieghts_21' , numba.float64[:,:]),
	('wieghts_22' , numba.float64[:,:]),
	('wieghts_23' , numba.float64[:,:]),
	('wieghts_24' , numba.float64[:,:]),
	('wieghts_25' , numba.float64[:,:]),
	('wieghts_26' , numba.float64[:,:]),
	('wieghts_27' , numba.float64[:,:]),
	('wieghts_28' , numba.float64[:,:]),
	('wieghts_29' , numba.float64[:,:]),
	('wieghts_30' , numba.float64[:,:]),
	('wieghts_31' , numba.float64[:,:]),
	('wieghts_32' , numba.float64[:,:]),
	('wieghts_33' , numba.float64[:,:]),
	('wieghts_34' , numba.float64[:,:]),
	('wieghts_35' , numba.float64[:,:]),
	('wieghts_36' , numba.float64[:,:]),
	('wieghts_37' , numba.float64[:,:]),
	('wieghts_38' , numba.float64[:,:]),
	('wieghts_39' , numba.float64[:,:]),
	('wieghts_40' , numba.float64[:,:]),
	('wieghts_41' , numba.float64[:,:]),
	('wieghts_42' , numba.float64[:,:]),
	('wieghts_43' , numba.float64[:,:]),
	('wieghts_44' , numba.float64[:,:]),
	('wieghts_45' , numba.float64[:,:]),
	('wieghts_46' , numba.float64[:,:]),
	('wieghts_47' , numba.float64[:,:]),
	('wieghts_48' , numba.float64[:,:]),
	('wieghts_49' , numba.float64[:,:]),
	('wieghts_50' , numba.float64[:,:]),
	('wieghts_51' , numba.float64[:,:]),
	('wieghts_52' , numba.float64[:,:]),
	('wieghts_53' , numba.float64[:,:]),
	('wieghts_54' , numba.float64[:,:]),
	('wieghts_55' , numba.float64[:,:]),
	('wieghts_s1' , numba.float64[:,:]),
	('wieghts_s2' , numba.float64[:,:]),
	('wieghts_s3' , numba.float64[:,:]),
	('wieghts_s4' , numba.float64[:,:]),
	('wieghts_s5' , numba.float64[:,:]),
	('wieghts_s6' , numba.float64[:,:]),
	('wieghts_s7' , numba.float64[:,:]),
	('wieghts_s8' , numba.float64[:,:]),
	('wieghts_s9' , numba.float64[:,:]),
	('wieghts_s10' , numba.float64[:,:]),
	('wieghts_s11' , numba.float64[:,:]),
	('wieghts_s12' , numba.float64[:,:]),
	('wieghts_s13' , numba.float64[:,:]),
	('wieghts_s14' , numba.float64[:,:]),
	('wieghts_s15' , numba.float64[:,:]),
	('wieghts_s16' , numba.float64[:,:]),
	('wieghts_s17' , numba.float64[:,:]),
	('wieghts_s18' , numba.float64[:,:]),
	('wieghts_s19' , numba.float64[:,:]),
	('wieghts_s20' , numba.float64[:,:]),
	('state' , numba.float64[:,:])])
class R_NeuralNet : # feed forward nn with 3 hidden layers maximum
	def __init__(self):
		self.fitness = 0.0 #initilliy 0 fitness
		self.accuracy= 0.0 #initially 0 accuracy 
		#adjacent fitness is the fitness of the creature nomalized to the maximum fitness in the generation
		#accuracy discribes the degree to which the outputs are exactly the same as refrences 
		self.NumOfHid_1 = random.randint(HIDDEN_1_MIN,HIDDEN_1_MAX) 
		self.NumOfHid_2 = random.randint(HIDDEN_2_MIN,HIDDEN_2_MAX) 
		self.NumOfHid_3 = random.randint(HIDDEN_3_MIN,HIDDEN_3_MAX) 
		self.NumOfHid_4 = random.randint(HIDDEN_4_MIN,HIDDEN_4_MAX) 
		self.NumOfHid_5 = random.randint(HIDDEN_5_MIN,HIDDEN_5_MAX) 
		self.NumOfHid_6 = random.randint(HIDDEN_6_MIN ,HIDDEN_6_MAX) 
		self.NumOfHid_7 = random.randint(HIDDEN_7_MIN,HIDDEN_7_MAX) 
		self.NumOfHid_8 = random.randint(HIDDEN_8_MIN,HIDDEN_8_MAX) 
		self.NumOfHid_9 = random.randint(HIDDEN_9_MIN,HIDDEN_9_MAX) 
		#start whith random number of hidden layer nodes
		self.NumOfStates = random.randint(STATE_MIN,STATE_MAX) # number of state variables for the system 
		#state of the system
		self.BaisIn = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(INPUT,1))
		self.BaisOut = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(OUTPUT,1))
		self.BaisHid_1 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_1,1))
		self.BaisHid_2 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_2,1))
		self.BaisHid_3 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_3,1))
		self.BaisHid_4 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_4,1))
		self.BaisHid_5 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_5,1))
		self.BaisHid_6 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_6,1))
		self.BaisHid_7 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_7,1))
		self.BaisHid_8 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_8,1))
		self.BaisHid_9 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(self.NumOfHid_9,1))
		#
		# wieghts
		#
		self.wieghts_1 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_1,INPUT))#from input to hidden_1
		self.wieghts_2 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_2,INPUT))#from input to hidden_2
		self.wieghts_3 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_3,INPUT))#from input to hidden_3
		self.wieghts_4 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_4,INPUT))#from input to hidden_4
		self.wieghts_5 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_5,INPUT))#from input to hidden_5
		self.wieghts_6 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,INPUT))#from input to hidden_6
		self.wieghts_7 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,INPUT))#from input to hidden_7
		self.wieghts_8 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,INPUT))#from input to hidden_8
		self.wieghts_9 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,INPUT))#from input to hidden_9
		self.wieghts_10 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,INPUT)) # from input directly to output
		#######################################################################
		self.wieghts_11 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_2,self.NumOfHid_1))#from hidden_1 to hidden_2
		self.wieghts_12 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_3,self.NumOfHid_1))#from hidden_1 to hidden_3
		self.wieghts_13 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_4,self.NumOfHid_1))#from hidden_1 to hidden_4
		self.wieghts_14 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_5,self.NumOfHid_1))#from hidden_1 to hidden_5
		self.wieghts_15 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,self.NumOfHid_1))#from hidden_1 to hidden_6
		self.wieghts_16 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfHid_1))#from hidden_1 to hidden_7
		self.wieghts_17 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_1))#from hidden_1 to hidden_8
		self.wieghts_18 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_1))#from hidden_1 to hidden_9
		self.wieghts_19 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_1))#from hidden_1 to output
		########################################################################
		self.wieghts_20 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_3,self.NumOfHid_2))#from hidden_2 to hidden_3
		self.wieghts_21 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_4,self.NumOfHid_2))#from hidden_2 to hidden_4
		self.wieghts_22 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_5,self.NumOfHid_2))#from hidden_2 to hidden_5
		self.wieghts_23 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,self.NumOfHid_2))#from hidden_2 to hidden_6
		self.wieghts_24 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfHid_2))#from hidden_2 to hidden_7
		self.wieghts_25 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_2))#from hidden_2 to hidden_8
		self.wieghts_26 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_2))#from hidden_2 to hidden_9
		self.wieghts_27 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_2))#from hidden_2 to output
		#########################################################################
		self.wieghts_28 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_4,self.NumOfHid_3))#from hidden_3 to hidden_4
		self.wieghts_29 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_5,self.NumOfHid_3))#from hidden_3 to hidden_5
		self.wieghts_30 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,self.NumOfHid_3))#from hidden_3 to hidden_6
		self.wieghts_31 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfHid_3))#from hidden_3 to hidden_7
		self.wieghts_32 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_3))#from hidden_3 to hidden_8
		self.wieghts_33 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_3))#from hidden_3 to hidden_9
		self.wieghts_34 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_3))#from hidden_3 to output
		##########################################################################
		self.wieghts_35 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_5,self.NumOfHid_4))#from hidden_4 to hidden_5
		self.wieghts_36 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,self.NumOfHid_4))#from hidden_4 to hidden_6
		self.wieghts_37 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfHid_4))#from hidden_4 to hidden_7
		self.wieghts_38 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_4))#from hidden_4 to hidden_8
		self.wieghts_39 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_4))#from hidden_4 to hidden_9
		self.wieghts_40 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_4))#from hidden_4 to output
		###########################################################################
		self.wieghts_41 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,self.NumOfHid_5))#from hidden_5 to hidden_6
		self.wieghts_42 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfHid_5))#from hidden_5 to hidden_7
		self.wieghts_43 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_5))#from hidden_5 to hidden_8
		self.wieghts_44 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_5))#from hidden_5 to hidden_9
		self.wieghts_45 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_5))#from hidden_5 to output
		###########################################################################
		self.wieghts_46 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfHid_6))#from hidden_6 to hidden_7
		self.wieghts_47 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_6))#from hidden_6 to hidden_8
		self.wieghts_48 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_6))#from hidden_6 to hidden_9
		self.wieghts_49 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_6))#from hidden_6 to output
		###########################################################################
		self.wieghts_50 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfHid_7))#from hidden_7 to hidden_8
		self.wieghts_51 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_7))#from hidden_7 to hidden_9
		self.wieghts_52 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_7))#from hidden_7 to output
		###########################################################################
		self.wieghts_53 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfHid_8))#from hidden_8 to hidden_9
		self.wieghts_54 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_8))#from hidden_8 to output
		###########################################################################
		self.wieghts_55 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfHid_9))#from hidden_9 to output
		###########################################################################
		self.wieghts_s1 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,INPUT)) # from input to the system state
		self.wieghts_s2 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_1))# from hidden_1 to the system state
		self.wieghts_s3 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_2)) #from hidden_2 to the system state
		self.wieghts_s4 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_3)) #from hidden_3 to the system state
		self.wieghts_s5 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_4))# from hidden_4 to the system state
		self.wieghts_s6 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_5)) #from hidden_5 to the system state
		self.wieghts_s7 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_6)) #from hidden_6 to the system state
		self.wieghts_s8 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_7))# from hidden_7 to the system state
		self.wieghts_s9 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_8)) #from hidden_8 to the system state
		self.wieghts_s10 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfStates,self.NumOfHid_9)) #from hidden_9 to the system state
		#############################################################################
		self.wieghts_s11 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_1,self.NumOfStates)) # feedback from the states to hidden_1
		self.wieghts_s12 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_2,self.NumOfStates)) # feedback from the states to hidden_2
		self.wieghts_s13 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_3,self.NumOfStates)) # feedback from the states to hidden_3
		self.wieghts_s14 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_4,self.NumOfStates)) # feedback from the states to hidden_4
		self.wieghts_s15 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_5,self.NumOfStates)) # feedback from the states to hidden_5
		self.wieghts_s16 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_6,self.NumOfStates)) # feedback from the states to hidden_6
		self.wieghts_s17 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_7,self.NumOfStates)) # feedback from the states to hidden_7
		self.wieghts_s18 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_8,self.NumOfStates)) # feedback from the states to hidden_8
		self.wieghts_s19 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(self.NumOfHid_9,self.NumOfStates)) # feedback from the states to hidden_9
		self.wieghts_s20 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT,self.NumOfStates)) # feedback from the states to OUTPUT
		#
		#state
		#
		self.state = np.zeros((self.NumOfStates,1))#every creature starts from state zero
		#
	# activation functions
	#relu 
	#@numba.njit(parallel = True)


	def ReLu (self,x):
		return np.maximum(x,0)
	#tanh
	def tanh(self,x):
		return np.tanh(x)
	#sigmoid
	def sigmoid(self,X):
   		return 1/(1+np.exp(-X))
   	#soft max
	def softmax(self,X):
		expo = np.exp(X)
		expo_sum = np.sum(np.exp(X))
		return expo/expo_sum
	#wall
	def wall(self,x):
		return x*1/((x-1)*(x-1)+1)
	#relu6
	def ReLu6(self,x):
			return np.minimum((np.maximum(x,0)),6)
	#silu
	def silu(self,x):
		return x/(1 + np.exp(-x))


	# very fast matrex by vector multiplication

	def propagate(self,A):
		#feed the input to the input layer and then observe the output
		input = np.reshape(A,(len(A),1)) 
		input = input + self.BaisIn
		hidden_1 =  np.dot(self.wieghts_1,input) + np.dot(self.wieghts_s11,self.state) + self.BaisHid_1
		#hid_1 = ReLu(W1*in + W_S11 *S+ Bhid1)
		hidden_1 = self.silu(hidden_1)
		#apply the activation function to the hidden_1 layer
		hidden_2 = np.dot(self.wieghts_2,input) + np.dot(self.wieghts_s12,self.state)
		hidden_2 = hidden_2  +  np.dot(self.wieghts_11,hidden_1)+ self.BaisHid_2
		#hid_2 = ReLu(W2*in + W_S12*S + W11*hid_1+ Bhid_2)
		hidden_2 = self.ReLu6(hidden_2)
		#apply the activation function to the hidden_2 layer
		hidden_3 =  np.dot(self.wieghts_3,input) + np.dot(self.wieghts_s13,self.state)
		hidden_3 = hidden_3 +  np.dot(self.wieghts_12,hidden_1) 
		hidden_3 = hidden_3 + np.dot(self.wieghts_20,hidden_2) + self.BaisHid_3 
		#hid_3 = ReLu(W3*in + W12*hid_1 + W20*hid_2 + W_S13*S + Bhid_3)
		hidden_3 = self.silu(hidden_3)
		#apply the activation function to the hidden_3 layer
		hidden_4 = np.dot(self.wieghts_4,input) + np.dot(self.wieghts_s14,self.state)
		hidden_4 = hidden_4 + np.dot(self.wieghts_13,hidden_1)
		hidden_4 = hidden_4 +  np.dot(self.wieghts_21,hidden_2)
		hidden_4 = hidden_4 +  np.dot(self.wieghts_28,hidden_3) + self.BaisHid_4
		hidden_4 = self.ReLu6(hidden_4)
		#apply the activation function to the hidden_4 layer
		hidden_5 = np.dot(self.wieghts_5,input) + np.dot(self.wieghts_s15,self.state)
		hidden_5 = hidden_5 + np.dot(self.wieghts_14,hidden_1)
		hidden_5 = hidden_5 + np.dot(self.wieghts_22,hidden_2)
		hidden_5 = hidden_5 + np.dot(self.wieghts_29,hidden_3)
		hidden_5 = hidden_5 + np.dot(self.wieghts_35,hidden_4) + self.BaisHid_5
		hidden_5 = self.silu(hidden_5)
		#apply the activation function to the hidden_5 layer
		hidden_6 = np.dot(self.wieghts_6,input) + np.dot(self.wieghts_s16,self.state)
		hidden_6 = hidden_6 + np.dot(self.wieghts_15,hidden_1)
		hidden_6 = hidden_6 + np.dot(self.wieghts_23,hidden_2)
		hidden_6 = hidden_6 + np.dot(self.wieghts_30,hidden_3)
		hidden_6 = hidden_6 + np.dot(self.wieghts_36,hidden_4)
		hidden_6 = hidden_6 + np.dot(self.wieghts_41,hidden_5) + self.BaisHid_6
		hidden_6 = self.tanh(hidden_6)
		#apply the activation function to the hidden_6 layer
		hidden_7 = np.dot(self.wieghts_7,input) + np.dot(self.wieghts_s17,self.state)
		hidden_7 = hidden_7 + np.dot(self.wieghts_16,hidden_1)
		hidden_7 = hidden_7 + np.dot(self.wieghts_24,hidden_2)
		hidden_7 = hidden_7 + np.dot(self.wieghts_31,hidden_3)
		hidden_7 = hidden_7 + np.dot(self.wieghts_37,hidden_4)
		hidden_7 = hidden_7 + np.dot(self.wieghts_42,hidden_5)
		hidden_7 = hidden_7 + np.dot(self.wieghts_46,hidden_6) + self.BaisHid_7
		hidden_7 = self.silu(hidden_7)
		#apply the activation function to the hidden_7 layer
		hidden_8 = np.dot(self.wieghts_8,input) + np.dot(self.wieghts_s18,self.state)
		hidden_8 = hidden_8 + np.dot(self.wieghts_17,hidden_1)
		hidden_8 = hidden_8 + np.dot(self.wieghts_25,hidden_2)
		hidden_8 = hidden_8 + np.dot(self.wieghts_32,hidden_3)
		hidden_8 = hidden_8 + np.dot(self.wieghts_38,hidden_4)
		hidden_8 = hidden_8 + np.dot(self.wieghts_43,hidden_5)
		hidden_8 = hidden_8 + np.dot(self.wieghts_47,hidden_6)
		hidden_8 = hidden_8 + np.dot(self.wieghts_50,hidden_7) + self.BaisHid_8
		hidden_8 = self.ReLu6(hidden_8)
		#apply the activation function to the hidden_8 layer
		hidden_9 = np.dot(self.wieghts_9,input) + np.dot(self.wieghts_s19,self.state)
		hidden_9 = hidden_9 + np.dot(self.wieghts_18,hidden_1)
		hidden_9 = hidden_9 + np.dot(self.wieghts_26,hidden_2)
		hidden_9 = hidden_9 + np.dot(self.wieghts_33,hidden_3)
		hidden_9 = hidden_9 + np.dot(self.wieghts_39,hidden_4)
		hidden_9 = hidden_9 + np.dot(self.wieghts_44,hidden_5)
		hidden_9 = hidden_9 + np.dot(self.wieghts_48,hidden_6)
		hidden_9 = hidden_9 + np.dot(self.wieghts_51,hidden_7)
		hidden_9 = hidden_9 + np.dot(self.wieghts_53,hidden_8) + self.BaisHid_9
		hidden_9 = self.ReLu6(hidden_9)
		#apply the activation function to the hidden_9 layer
		output = np.dot(self.wieghts_10,input) + np.dot(self.wieghts_s20,self.state)
		output = output + np.dot(self.wieghts_19,hidden_1)
		output = output + np.dot(self.wieghts_27,hidden_2)
		output = output + np.dot(self.wieghts_34,hidden_3)
		output = output + np.dot(self.wieghts_40,hidden_4)
		output = output + np.dot(self.wieghts_45,hidden_5)
		output = output + np.dot(self.wieghts_49,hidden_6)
		output = output + np.dot(self.wieghts_52,hidden_7)
		output = output + np.dot(self.wieghts_54,hidden_8)
		output = output + np.dot(self.wieghts_55,hidden_9) + self.BaisOut
		output = self.softmax(output)
		#apply the activation function to the output layer
		self.state = np.dot(self.wieghts_s1,input)
		self.state = self.state + np.dot(self.wieghts_s2,hidden_1)
		self.state = self.state + np.dot(self.wieghts_s3,hidden_2) 
		self.state = self.state + np.dot(self.wieghts_s4,hidden_3)
		self.state = self.state + np.dot(self.wieghts_s5,hidden_4)
		self.state = self.state + np.dot(self.wieghts_s6,hidden_5) 
		self.state = self.state + np.dot(self.wieghts_s7,hidden_6)
		self.state = self.state + np.dot(self.wieghts_s8,hidden_7)
		self.state = self.state + np.dot(self.wieghts_s9,hidden_8) 
		self.state = self.state + np.dot(self.wieghts_s10,hidden_9)
		self.state = self.tanh(self.state) 
		return (output)
	def mutate(self):
		mutated_creature = self.copy()
		#the new mutated creature
		# mutate the elements of w1,w2,w3 ...w55 and ws1...ws20 every element has a chance = mutation rate of mutate 
		mutated_creature.wieghts_1 = self.MuTaTe(mutated_creature.wieghts_1,MUTATION_RATE*(1 + 4*MUTATION_DELTA))
		mutated_creature.wieghts_2 = self.MuTaTe(mutated_creature.wieghts_2,MUTATION_RATE*(1 + 3*MUTATION_DELTA))
		mutated_creature.wieghts_3 = self.MuTaTe(mutated_creature.wieghts_3,MUTATION_RATE*(1 + 2*MUTATION_DELTA))
		mutated_creature.wieghts_4 = self.MuTaTe(mutated_creature.wieghts_4,MUTATION_RATE*(1 + MUTATION_DELTA))
		mutated_creature.wieghts_5 = self.MuTaTe(mutated_creature.wieghts_5,MUTATION_RATE)
		mutated_creature.wieghts_6 = self.MuTaTe(mutated_creature.wieghts_6,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.wieghts_7 = self.MuTaTe(mutated_creature.wieghts_7,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_8 = self.MuTaTe(mutated_creature.wieghts_8,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_9 = self.MuTaTe(mutated_creature.wieghts_9,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_10 = self.MuTaTe(mutated_creature.wieghts_10,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_11 = self.MuTaTe(mutated_creature.wieghts_11,MUTATION_RATE*(1 + 3*MUTATION_DELTA))
		mutated_creature.wieghts_12 = self.MuTaTe(mutated_creature.wieghts_12,MUTATION_RATE*(1 + 2*MUTATION_DELTA))
		mutated_creature.wieghts_13 = self.MuTaTe(mutated_creature.wieghts_13,MUTATION_RATE*(1 + MUTATION_DELTA))
		mutated_creature.wieghts_14 = self.MuTaTe(mutated_creature.wieghts_14,MUTATION_RATE)
		mutated_creature.wieghts_15 = self.MuTaTe(mutated_creature.wieghts_15,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.wieghts_16 = self.MuTaTe(mutated_creature.wieghts_16,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_17 = self.MuTaTe(mutated_creature.wieghts_17,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_18 = self.MuTaTe(mutated_creature.wieghts_18,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_19 = self.MuTaTe(mutated_creature.wieghts_19,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_20 = self.MuTaTe(mutated_creature.wieghts_20,MUTATION_RATE*(1 + 2*MUTATION_DELTA))
		mutated_creature.wieghts_21 = self.MuTaTe(mutated_creature.wieghts_21,MUTATION_RATE*(1 + MUTATION_DELTA))
		mutated_creature.wieghts_22 = self.MuTaTe(mutated_creature.wieghts_22,MUTATION_RATE)
		mutated_creature.wieghts_23 = self.MuTaTe(mutated_creature.wieghts_23,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.wieghts_24 = self.MuTaTe(mutated_creature.wieghts_24,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_25 = self.MuTaTe(mutated_creature.wieghts_25,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_26 = self.MuTaTe(mutated_creature.wieghts_26,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_27 = self.MuTaTe(mutated_creature.wieghts_27,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_28 = self.MuTaTe(mutated_creature.wieghts_28,MUTATION_RATE*(1 + MUTATION_DELTA))
		mutated_creature.wieghts_29 = self.MuTaTe(mutated_creature.wieghts_29,MUTATION_RATE)
		mutated_creature.wieghts_30 = self.MuTaTe(mutated_creature.wieghts_30,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.wieghts_31 = self.MuTaTe(mutated_creature.wieghts_31,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_32 = self.MuTaTe(mutated_creature.wieghts_32,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_33 = self.MuTaTe(mutated_creature.wieghts_33,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_34 = self.MuTaTe(mutated_creature.wieghts_34,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_35 = self.MuTaTe(mutated_creature.wieghts_35,MUTATION_RATE)
		mutated_creature.wieghts_36 = self.MuTaTe(mutated_creature.wieghts_36,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.wieghts_37 = self.MuTaTe(mutated_creature.wieghts_37,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_38 = self.MuTaTe(mutated_creature.wieghts_38,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_39 = self.MuTaTe(mutated_creature.wieghts_39,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_40 = self.MuTaTe(mutated_creature.wieghts_40,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_41 = self.MuTaTe(mutated_creature.wieghts_41,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.wieghts_42 = self.MuTaTe(mutated_creature.wieghts_42,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_43 = self.MuTaTe(mutated_creature.wieghts_43,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_44 = self.MuTaTe(mutated_creature.wieghts_44,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_45 = self.MuTaTe(mutated_creature.wieghts_45,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_46 = self.MuTaTe(mutated_creature.wieghts_46,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.wieghts_47 = self.MuTaTe(mutated_creature.wieghts_47,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_48 = self.MuTaTe(mutated_creature.wieghts_48,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_49 = self.MuTaTe(mutated_creature.wieghts_49,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_50 = self.MuTaTe(mutated_creature.wieghts_50,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.wieghts_51 = self.MuTaTe(mutated_creature.wieghts_51,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_52 = self.MuTaTe(mutated_creature.wieghts_52,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_53 = self.MuTaTe(mutated_creature.wieghts_53,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		mutated_creature.wieghts_54 = self.MuTaTe(mutated_creature.wieghts_54,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_55 = self.MuTaTe(mutated_creature.wieghts_55,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.wieghts_s1 = self.MuTaTe(mutated_creature.wieghts_s1,MUTATION_RATE)
		mutated_creature.wieghts_s2 = self.MuTaTe(mutated_creature.wieghts_s2,MUTATION_RATE)
		mutated_creature.wieghts_s3 = self.MuTaTe(mutated_creature.wieghts_s3,MUTATION_RATE)
		mutated_creature.wieghts_s4 = self.MuTaTe(mutated_creature.wieghts_s4,MUTATION_RATE)
		mutated_creature.wieghts_s5 = self.MuTaTe(mutated_creature.wieghts_s5,MUTATION_RATE)
		mutated_creature.wieghts_s6 = self.MuTaTe(mutated_creature.wieghts_s6,MUTATION_RATE)
		mutated_creature.wieghts_s7 = self.MuTaTe(mutated_creature.wieghts_s7,MUTATION_RATE)
		mutated_creature.wieghts_s8 = self.MuTaTe(mutated_creature.wieghts_s8,MUTATION_RATE)
		mutated_creature.wieghts_s9 = self.MuTaTe(mutated_creature.wieghts_s9,MUTATION_RATE)
		mutated_creature.wieghts_s10 = self.MuTaTe(mutated_creature.wieghts_s10,MUTATION_RATE)
		mutated_creature.wieghts_s11 = self.MuTaTe(mutated_creature.wieghts_s11,MUTATION_RATE)
		mutated_creature.wieghts_s12 = self.MuTaTe(mutated_creature.wieghts_s12,MUTATION_RATE)
		mutated_creature.wieghts_s13 = self.MuTaTe(mutated_creature.wieghts_s13,MUTATION_RATE)
		mutated_creature.wieghts_s14 = self.MuTaTe(mutated_creature.wieghts_s14,MUTATION_RATE)
		mutated_creature.wieghts_s15 = self.MuTaTe(mutated_creature.wieghts_s15,MUTATION_RATE)
		mutated_creature.wieghts_s16 = self.MuTaTe(mutated_creature.wieghts_s16,MUTATION_RATE)
		mutated_creature.wieghts_s17 = self.MuTaTe(mutated_creature.wieghts_s17,MUTATION_RATE)
		mutated_creature.wieghts_s18 = self.MuTaTe(mutated_creature.wieghts_s18,MUTATION_RATE)
		mutated_creature.wieghts_s19 = self.MuTaTe(mutated_creature.wieghts_s19,MUTATION_RATE)
		mutated_creature.wieghts_s20 = self.MuTaTe(mutated_creature.wieghts_s20,MUTATION_RATE)
		mutated_creature.BaisIn = self.MuTaTe(mutated_creature.BaisIn,MUTATION_RATE*(1 + +5*MUTATION_DELTA))	
		mutated_creature.BaisOut = self.MuTaTe(mutated_creature.BaisOut,MUTATION_RATE*(1 + -5*MUTATION_DELTA))
		mutated_creature.BaisHid_1 = self.MuTaTe(mutated_creature.BaisHid_1,MUTATION_RATE*(1 + 4*MUTATION_DELTA))
		mutated_creature.BaisHid_2 = self.MuTaTe(mutated_creature.BaisHid_2,MUTATION_RATE*(1 + 3*MUTATION_DELTA))
		mutated_creature.BaisHid_3 = self.MuTaTe(mutated_creature.BaisHid_3,MUTATION_RATE*(1 + 2*MUTATION_DELTA))
		mutated_creature.BaisHid_4 = self.MuTaTe(mutated_creature.BaisHid_4,MUTATION_RATE*(1 + 1*MUTATION_DELTA))
		mutated_creature.BaisHid_5 = self.MuTaTe(mutated_creature.BaisHid_5,MUTATION_RATE)
		mutated_creature.BaisHid_6 = self.MuTaTe(mutated_creature.BaisHid_6,MUTATION_RATE*(1 + -1*MUTATION_DELTA))
		mutated_creature.BaisHid_7 = self.MuTaTe(mutated_creature.BaisHid_7,MUTATION_RATE*(1 + -2*MUTATION_DELTA))
		mutated_creature.BaisHid_8 = self.MuTaTe(mutated_creature.BaisHid_8,MUTATION_RATE*(1 + -3*MUTATION_DELTA))
		mutated_creature.BaisHid_9 = self.MuTaTe(mutated_creature.BaisHid_9,MUTATION_RATE*(1 + -4*MUTATION_DELTA))
		return(mutated_creature)
	def reset(self):
		self.state = np.zeros((self.NumOfStates,1))
		return 1
	def MuTaTe(self,mat,mut_rate):
		num_rows , num_cols = mat.shape
		mask = np.random.binomial(1, mut_rate , size=(num_rows,num_cols))
		delta = np.multiply(np.random.uniform(-MUT_ACC,MUT_ACC,(num_rows,num_cols)),mask)
		return(mat + delta)					
	def crossOver(self , R_NeuralNet ):
		#here two nn are mixed together to form new nn
		crossover_creature = self.copy()
		if (crossover_creature.NumOfHid_1 == R_NeuralNet.NumOfHid_1):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_1 = R_NeuralNet.wieghts_1
			if((random.randint(0,100) > 50 ) and (crossover_creature.NumOfHid_2 == R_NeuralNet.NumOfHid_2)):
				crossover_creature.wieghts_11 = R_NeuralNet.wieghts_11
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_3 == R_NeuralNet.NumOfHid_3):
				crossover_creature.wieghts_12 = R_NeuralNet.wieghts_12
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_4 == R_NeuralNet.NumOfHid_4):
				crossover_creature.wieghts_13 = R_NeuralNet.wieghts_13
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
				crossover_creature.wieghts_14 = R_NeuralNet.wieghts_14
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_15 = R_NeuralNet.wieghts_15
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_16 = R_NeuralNet.wieghts_16
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_17 = R_NeuralNet.wieghts_17
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_18 = R_NeuralNet.wieghts_18
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_19 = R_NeuralNet.wieghts_19

		#############################################################################################
		if (crossover_creature.NumOfHid_2 == R_NeuralNet.NumOfHid_2):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_2 = R_NeuralNet.wieghts_2
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_3 == R_NeuralNet.NumOfHid_3):
				crossover_creature.wieghts_20 = R_NeuralNet.wieghts_20
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_4 == R_NeuralNet.NumOfHid_4):
				crossover_creature.wieghts_21 = R_NeuralNet.wieghts_21
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
				crossover_creature.wieghts_22 = R_NeuralNet.wieghts_22
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_23 = R_NeuralNet.wieghts_23
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_24 = R_NeuralNet.wieghts_24
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_25 = R_NeuralNet.wieghts_25
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_26 = R_NeuralNet.wieghts_26
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_27 = R_NeuralNet.wieghts_27
		#############################################################################################
		if (crossover_creature.NumOfHid_3 == R_NeuralNet.NumOfHid_3):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_3 = R_NeuralNet.wieghts_3
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_4 == R_NeuralNet.NumOfHid_4):
				crossover_creature.wieghts_28 = R_NeuralNet.wieghts_28
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
				crossover_creature.wieghts_29 = R_NeuralNet.wieghts_29
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_30 = R_NeuralNet.wieghts_30
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_31 = R_NeuralNet.wieghts_31
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_32 = R_NeuralNet.wieghts_32
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_33 = R_NeuralNet.wieghts_33
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_34 = R_NeuralNet.wieghts_34
		#############################################################################################
		if (crossover_creature.NumOfHid_4 == R_NeuralNet.NumOfHid_4):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_4 = R_NeuralNet.wieghts_4
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
				crossover_creature.wieghts_35 = R_NeuralNet.wieghts_35
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_36 = R_NeuralNet.wieghts_36
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_37 = R_NeuralNet.wieghts_37
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_38 = R_NeuralNet.wieghts_38
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_39 = R_NeuralNet.wieghts_39
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_40 = R_NeuralNet.wieghts_40
		#############################################################################################
		if (crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_5 = R_NeuralNet.wieghts_5
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_41 = R_NeuralNet.wieghts_41
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_42 = R_NeuralNet.wieghts_42
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_43 = R_NeuralNet.wieghts_43
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_44 = R_NeuralNet.wieghts_44
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_45 = R_NeuralNet.wieghts_45
		#############################################################################################
		if (crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_6 = R_NeuralNet.wieghts_6
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_46 = R_NeuralNet.wieghts_46
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_47 = R_NeuralNet.wieghts_47
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_48 = R_NeuralNet.wieghts_48
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_49 = R_NeuralNet.wieghts_49
		#############################################################################################
		if (crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_7 = R_NeuralNet.wieghts_7
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_50 = R_NeuralNet.wieghts_50
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_51 = R_NeuralNet.wieghts_51
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_52 = R_NeuralNet.wieghts_52
		#############################################################################################
		if (crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_8 = R_NeuralNet.wieghts_8
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_53 = R_NeuralNet.wieghts_53
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_54 = R_NeuralNet.wieghts_54
		#############################################################################################
		if (crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_9 = R_NeuralNet.wieghts_9
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_55 = R_NeuralNet.wieghts_55
		#############################################################################################
		if (crossover_creature.NumOfStates == R_NeuralNet.NumOfStates):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_s1 = R_NeuralNet.wieghts_s1
			if((random.randint(0,100) > 50 ) and (crossover_creature.NumOfHid_1 == R_NeuralNet.NumOfHid_1)):
				crossover_creature.wieghts_s2 = R_NeuralNet.wieghts_s2
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_2 == R_NeuralNet.NumOfHid_2):
				crossover_creature.wieghts_s3 = R_NeuralNet.wieghts_s3
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_3 == R_NeuralNet.NumOfHid_3):
				crossover_creature.wieghts_s4 = R_NeuralNet.wieghts_s4
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_4 == R_NeuralNet.NumOfHid_4):
				crossover_creature.wieghts_s5 = R_NeuralNet.wieghts_s5
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
				crossover_creature.wieghts_s6 = R_NeuralNet.wieghts_s6
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_s7 = R_NeuralNet.wieghts_s7
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_s8 = R_NeuralNet.wieghts_s8
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_s9 = R_NeuralNet.wieghts_s9
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9):
				crossover_creature.wieghts_s10 = R_NeuralNet.wieghts_s10
		#############################################################################################
		if (crossover_creature.NumOfStates == R_NeuralNet.NumOfStates):
			if(random.randint(0,100) > 50):
				crossover_creature.wieghts_s20 = R_NeuralNet.wieghts_s20
			if((random.randint(0,100) > 50 ) and (crossover_creature.NumOfHid_9 == R_NeuralNet.NumOfHid_9)):
				crossover_creature.wieghts_s19 = R_NeuralNet.wieghts_s19
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_8 == R_NeuralNet.NumOfHid_8):
				crossover_creature.wieghts_s18 = R_NeuralNet.wieghts_s18
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_7 == R_NeuralNet.NumOfHid_7):
				crossover_creature.wieghts_s17 = R_NeuralNet.wieghts_s17
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_6 == R_NeuralNet.NumOfHid_6):
				crossover_creature.wieghts_s16 = R_NeuralNet.wieghts_s16
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_5 == R_NeuralNet.NumOfHid_5):
				crossover_creature.wieghts_s15 = R_NeuralNet.wieghts_s15
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_4 == R_NeuralNet.NumOfHid_4):
				crossover_creature.wieghts_s14 = R_NeuralNet.wieghts_s14
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_3 == R_NeuralNet.NumOfHid_3):
				crossover_creature.wieghts_s13 = R_NeuralNet.wieghts_s13
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_2 == R_NeuralNet.NumOfHid_2):
				crossover_creature.wieghts_s12 = R_NeuralNet.wieghts_s12
			if(random.randint(0,100) > 50 and crossover_creature.NumOfHid_1 == R_NeuralNet.NumOfHid_1):
				crossover_creature.wieghts_s11 = R_NeuralNet.wieghts_s11
		#############################################################################################
		crossover_creature.reset()
		return(crossover_creature)
	def zeroCreature(self):
		self.BaisIn = np.zeros((INPUT,1))
		self.BaisOut = np.zeros((OUTPUT,1))
		self.BaisHid_1 = np.zeros((self.NumOfHid_1,1))
		self.BaisHid_2 = np.zeros((self.NumOfHid_2,1))
		self.BaisHid_3 = np.zeros((self.NumOfHid_3,1))
		self.BaisHid_4 = np.zeros((self.NumOfHid_4,1))
		self.BaisHid_5 = np.zeros((self.NumOfHid_5,1))
		self.BaisHid_6 = np.zeros((self.NumOfHid_6,1))
		self.BaisHid_7 = np.zeros((self.NumOfHid_7,1))
		self.BaisHid_8 = np.zeros((self.NumOfHid_8,1))
		self.BaisHid_9 = np.zeros((self.NumOfHid_9,1))
		#
		# wieghts
		#
		self.wieghts_1 = np.zeros((self.NumOfHid_1,INPUT))#from input to hidden_1
		self.wieghts_2 = np.zeros((self.NumOfHid_2,INPUT))#from input to hidden_2
		self.wieghts_3 = np.zeros((self.NumOfHid_3,INPUT))#from input to hidden_3
		self.wieghts_4 = np.zeros((self.NumOfHid_4,INPUT))#from input to hidden_4
		self.wieghts_5 = np.zeros((self.NumOfHid_5,INPUT))#from input to hidden_5
		self.wieghts_6 = np.zeros((self.NumOfHid_6,INPUT))#from input to hidden_6
		self.wieghts_7 = np.zeros((self.NumOfHid_7,INPUT))#from input to hidden_7
		self.wieghts_8 = np.zeros((self.NumOfHid_8,INPUT))#from input to hidden_8
		self.wieghts_9 = np.zeros((self.NumOfHid_9,INPUT))#from input to hidden_9
		self.wieghts_10 = np.zeros((OUTPUT,INPUT)) # from input directly to output
		#######################################################################
		self.wieghts_11 = np.zeros((self.NumOfHid_2,self.NumOfHid_1))#from hidden_1 to hidden_2
		self.wieghts_12 = np.zeros((self.NumOfHid_3,self.NumOfHid_1))#from hidden_1 to hidden_3
		self.wieghts_13 = np.zeros((self.NumOfHid_4,self.NumOfHid_1))#from hidden_1 to hidden_4
		self.wieghts_14 = np.zeros((self.NumOfHid_5,self.NumOfHid_1))#from hidden_1 to hidden_5
		self.wieghts_15 = np.zeros((self.NumOfHid_6,self.NumOfHid_1))#from hidden_1 to hidden_6
		self.wieghts_16 = np.zeros((self.NumOfHid_7,self.NumOfHid_1))#from hidden_1 to hidden_7
		self.wieghts_17 = np.zeros((self.NumOfHid_8,self.NumOfHid_1))#from hidden_1 to hidden_8
		self.wieghts_18 = np.zeros((self.NumOfHid_9,self.NumOfHid_1))#from hidden_1 to hidden_9
		self.wieghts_19 = np.zeros((OUTPUT,self.NumOfHid_1))#from hidden_1 to output
		########################################################################
		self.wieghts_20 = np.zeros((self.NumOfHid_3,self.NumOfHid_2))#from hidden_2 to hidden_3
		self.wieghts_21 = np.zeros((self.NumOfHid_4,self.NumOfHid_2))#from hidden_2 to hidden_4
		self.wieghts_22 = np.zeros((self.NumOfHid_5,self.NumOfHid_2))#from hidden_2 to hidden_5
		self.wieghts_23 = np.zeros((self.NumOfHid_6,self.NumOfHid_2))#from hidden_2 to hidden_6
		self.wieghts_24 = np.zeros((self.NumOfHid_7,self.NumOfHid_2))#from hidden_2 to hidden_7
		self.wieghts_25 = np.zeros((self.NumOfHid_8,self.NumOfHid_2))#from hidden_2 to hidden_8
		self.wieghts_26 = np.zeros((self.NumOfHid_9,self.NumOfHid_2))#from hidden_2 to hidden_9
		self.wieghts_27 = np.zeros((OUTPUT,self.NumOfHid_2))#from hidden_2 to output
		#########################################################################
		self.wieghts_28 = np.zeros((self.NumOfHid_4,self.NumOfHid_3))#from hidden_3 to hidden_4
		self.wieghts_29 = np.zeros((self.NumOfHid_5,self.NumOfHid_3))#from hidden_3 to hidden_5
		self.wieghts_30 = np.zeros((self.NumOfHid_6,self.NumOfHid_3))#from hidden_3 to hidden_6
		self.wieghts_31 = np.zeros((self.NumOfHid_7,self.NumOfHid_3))#from hidden_3 to hidden_7
		self.wieghts_32 = np.zeros((self.NumOfHid_8,self.NumOfHid_3))#from hidden_3 to hidden_8
		self.wieghts_33 = np.zeros((self.NumOfHid_9,self.NumOfHid_3))#from hidden_3 to hidden_9
		self.wieghts_34 = np.zeros((OUTPUT,self.NumOfHid_3))#from hidden_3 to output
		##########################################################################
		self.wieghts_35 = np.zeros((self.NumOfHid_5,self.NumOfHid_4))#from hidden_4 to hidden_5
		self.wieghts_36 = np.zeros((self.NumOfHid_6,self.NumOfHid_4))#from hidden_4 to hidden_6
		self.wieghts_37 = np.zeros((self.NumOfHid_7,self.NumOfHid_4))#from hidden_4 to hidden_7
		self.wieghts_38 = np.zeros((self.NumOfHid_8,self.NumOfHid_4))#from hidden_4 to hidden_8
		self.wieghts_39 = np.zeros((self.NumOfHid_9,self.NumOfHid_4))#from hidden_4 to hidden_9
		self.wieghts_40 = np.zeros((OUTPUT,self.NumOfHid_4))#from hidden_4 to output
		###########################################################################
		self.wieghts_41 = np.zeros((self.NumOfHid_6,self.NumOfHid_5))#from hidden_5 to hidden_6
		self.wieghts_42 = np.zeros((self.NumOfHid_7,self.NumOfHid_5))#from hidden_5 to hidden_7
		self.wieghts_43 = np.zeros((self.NumOfHid_8,self.NumOfHid_5))#from hidden_5 to hidden_8
		self.wieghts_44 = np.zeros((self.NumOfHid_9,self.NumOfHid_5))#from hidden_5 to hidden_9
		self.wieghts_45 = np.zeros((OUTPUT,self.NumOfHid_5))#from hidden_5 to output
		###########################################################################
		self.wieghts_46 = np.zeros((self.NumOfHid_7,self.NumOfHid_6))#from hidden_6 to hidden_7
		self.wieghts_47 = np.zeros((self.NumOfHid_8,self.NumOfHid_6))#from hidden_6 to hidden_8
		self.wieghts_48 = np.zeros((self.NumOfHid_9,self.NumOfHid_6))#from hidden_6 to hidden_9
		self.wieghts_49 = np.zeros((OUTPUT,self.NumOfHid_6))#from hidden_6 to output
		###########################################################################
		self.wieghts_50 = np.zeros((self.NumOfHid_8,self.NumOfHid_7))#from hidden_7 to hidden_8
		self.wieghts_51 = np.zeros((self.NumOfHid_9,self.NumOfHid_7))#from hidden_7 to hidden_9
		self.wieghts_52 = np.zeros((OUTPUT,self.NumOfHid_7))#from hidden_7 to output
		###########################################################################
		self.wieghts_53 = np.zeros((self.NumOfHid_9,self.NumOfHid_8))#from hidden_8 to hidden_9
		self.wieghts_54 = np.zeros((OUTPUT,self.NumOfHid_8))#from hidden_8 to output
		###########################################################################
		self.wieghts_55 = np.zeros((OUTPUT,self.NumOfHid_9))#from hidden_9 to output
		###########################################################################
		self.wieghts_s1 = np.zeros((self.NumOfStates,INPUT)) # from input to the system state
		self.wieghts_s2 = np.zeros((self.NumOfStates,self.NumOfHid_1))# from hidden_1 to the system state
		self.wieghts_s3 = np.zeros((self.NumOfStates,self.NumOfHid_2)) #from hidden_2 to the system state
		self.wieghts_s4 = np.zeros((self.NumOfStates,self.NumOfHid_3)) #from hidden_3 to the system state
		self.wieghts_s5 = np.zeros((self.NumOfStates,self.NumOfHid_4))# from hidden_4 to the system state
		self.wieghts_s6 = np.zeros((self.NumOfStates,self.NumOfHid_5)) #from hidden_5 to the system state
		self.wieghts_s7 = np.zeros((self.NumOfStates,self.NumOfHid_6)) #from hidden_6 to the system state
		self.wieghts_s8 = np.zeros((self.NumOfStates,self.NumOfHid_7))# from hidden_7 to the system state
		self.wieghts_s9 = np.zeros((self.NumOfStates,self.NumOfHid_8)) #from hidden_8 to the system state
		self.wieghts_s10 = np.zeros((self.NumOfStates,self.NumOfHid_9)) #from hidden_9 to the system state
		#############################################################################
		self.wieghts_s11 = np.zeros((self.NumOfHid_1,self.NumOfStates)) # feedback from the states to hidden_1
		self.wieghts_s12 = np.zeros((self.NumOfHid_2,self.NumOfStates)) # feedback from the states to hidden_2
		self.wieghts_s13 = np.zeros((self.NumOfHid_3,self.NumOfStates)) # feedback from the states to hidden_3
		self.wieghts_s14 = np.zeros((self.NumOfHid_4,self.NumOfStates)) # feedback from the states to hidden_4
		self.wieghts_s15 = np.zeros((self.NumOfHid_5,self.NumOfStates)) # feedback from the states to hidden_5
		self.wieghts_s16 = np.zeros((self.NumOfHid_6,self.NumOfStates)) # feedback from the states to hidden_6
		self.wieghts_s17 = np.zeros((self.NumOfHid_7,self.NumOfStates)) # feedback from the states to hidden_7
		self.wieghts_s18 = np.zeros((self.NumOfHid_8,self.NumOfStates)) # feedback from the states to hidden_8
		self.wieghts_s19 = np.zeros((self.NumOfHid_9,self.NumOfStates)) # feedback from the states to hidden_9
		self.wieghts_s20 = np.zeros((OUTPUT,self.NumOfStates)) # feedback from the states to OUTPUT
		#
		return(self)
	def showStructure(self):
		print()
		print("input :" + str(INPUT))
		print("hidden layer 1 :"+str(self.NumOfHid_1))
		print("hidden layer 2 :"+str(self.NumOfHid_2))
		print("hidden layer 3 :"+str(self.NumOfHid_3))
		print("hidden layer 4 :"+str(self.NumOfHid_4))
		print("hidden layer 5 :"+str(self.NumOfHid_5))
		print("hidden layer 6 :"+str(self.NumOfHid_6))
		print("hidden layer 7 :"+str(self.NumOfHid_7))
		print("hidden layer 8 :"+str(self.NumOfHid_8))
		print("hidden layer 9 :"+str(self.NumOfHid_9))
		print("number of state variables :"+str(self.NumOfStates))
		print("output :" + str(OUTPUT))
	def copy(self):
		c = R_NeuralNet()
		c.NumOfHid_1 = self.NumOfHid_1
		c.NumOfHid_2 = self.NumOfHid_2
		c.NumOfHid_3 = self.NumOfHid_3
		c.NumOfHid_4 = self.NumOfHid_4
		c.NumOfHid_5 = self.NumOfHid_5
		c.NumOfHid_6 = self.NumOfHid_6
		c.NumOfHid_7 = self.NumOfHid_7
		c.NumOfHid_8 = self.NumOfHid_8
		c.NumOfHid_9 = self.NumOfHid_9
		c.NumOfStates = self.NumOfStates
		##############################
		c.state = np.zeros((c.NumOfStates,1))
		##############################
		c.BaisIn = self.BaisIn
		c.BaisHid_1 = self.BaisHid_1
		c.BaisHid_2 = self.BaisHid_2
		c.BaisHid_3 = self.BaisHid_3
		c.BaisHid_4 = self.BaisHid_4
		c.BaisHid_5 = self.BaisHid_5
		c.BaisHid_6 = self.BaisHid_6
		c.BaisHid_7 = self.BaisHid_7
		c.BaisHid_8 = self.BaisHid_8
		c.BaisHid_9 = self.BaisHid_9
		c.BaisOut = self.BaisOut
		#############################
		c.wieghts_1 = self.wieghts_1
		c.wieghts_2 = self.wieghts_2
		c.wieghts_3 = self.wieghts_3
		c.wieghts_4 = self.wieghts_4
		c.wieghts_5 = self.wieghts_5
		c.wieghts_6 = self.wieghts_6
		c.wieghts_7 = self.wieghts_7
		c.wieghts_8 = self.wieghts_8
		c.wieghts_9 = self.wieghts_9
		c.wieghts_10 = self.wieghts_10
		c.wieghts_11 = self.wieghts_11
		c.wieghts_12 = self.wieghts_12
		c.wieghts_13 = self.wieghts_13
		c.wieghts_14 = self.wieghts_14
		c.wieghts_15 = self.wieghts_15
		c.wieghts_16 = self.wieghts_16
		c.wieghts_17 = self.wieghts_17
		c.wieghts_18 = self.wieghts_18
		c.wieghts_19 = self.wieghts_19
		c.wieghts_20 = self.wieghts_20
		c.wieghts_21 = self.wieghts_21
		c.wieghts_22 = self.wieghts_22
		c.wieghts_23 = self.wieghts_23
		c.wieghts_24 = self.wieghts_24
		c.wieghts_25 = self.wieghts_25
		c.wieghts_26 = self.wieghts_26
		c.wieghts_27 = self.wieghts_27
		c.wieghts_28 = self.wieghts_28
		c.wieghts_29 = self.wieghts_29
		c.wieghts_30 = self.wieghts_30
		c.wieghts_31 = self.wieghts_31
		c.wieghts_32 = self.wieghts_32
		c.wieghts_33 = self.wieghts_33
		c.wieghts_34 = self.wieghts_34
		c.wieghts_35 = self.wieghts_35
		c.wieghts_36 = self.wieghts_36
		c.wieghts_37 = self.wieghts_37
		c.wieghts_38 = self.wieghts_38
		c.wieghts_39 = self.wieghts_39
		c.wieghts_40 = self.wieghts_40
		c.wieghts_41 = self.wieghts_41
		c.wieghts_42 = self.wieghts_42
		c.wieghts_43 = self.wieghts_43
		c.wieghts_44 = self.wieghts_44
		c.wieghts_45 = self.wieghts_45
		c.wieghts_46 = self.wieghts_46
		c.wieghts_47 = self.wieghts_47
		c.wieghts_48 = self.wieghts_48
		c.wieghts_49 = self.wieghts_49
		c.wieghts_50 = self.wieghts_50
		c.wieghts_51 = self.wieghts_51
		c.wieghts_52 = self.wieghts_52
		c.wieghts_53 = self.wieghts_53
		c.wieghts_54 = self.wieghts_54
		c.wieghts_55 = self.wieghts_55
		##############################
		c.wieghts_s1 = self.wieghts_s1
		c.wieghts_s2 = self.wieghts_s2
		c.wieghts_s3 = self.wieghts_s3
		c.wieghts_s4 = self.wieghts_s4
		c.wieghts_s5 = self.wieghts_s5
		c.wieghts_s6 = self.wieghts_s6
		c.wieghts_s7 = self.wieghts_s7
		c.wieghts_s8 = self.wieghts_s8
		c.wieghts_s9 = self.wieghts_s9
		c.wieghts_s10 = self.wieghts_s10
		c.wieghts_s11 = self.wieghts_s11
		c.wieghts_s12 = self.wieghts_s12
		c.wieghts_s13 = self.wieghts_s13
		c.wieghts_s14 = self.wieghts_s14
		c.wieghts_s15 = self.wieghts_s15
		c.wieghts_s16 = self.wieghts_s16
		c.wieghts_s17 = self.wieghts_s17
		c.wieghts_s18 = self.wieghts_s18
		c.wieghts_s19 = self.wieghts_s19
		c.wieghts_s20 = self.wieghts_s20
		return(c)



# store the parameters of the neural network.
simple method to save the parameters into two files , Architecure file describes the number of neurons in each layer.
wieghts file contains the actual values of those parameters

In [ ]:
def save(R_NeuralNet):
	Architecture = { 'INPUT' : INPUT ,
	'NumOfHid_1' : R_NeuralNet.NumOfHid_1 ,
	'NumOfHid_2' : R_NeuralNet.NumOfHid_2 ,
	'NumOfHid_3' : R_NeuralNet.NumOfHid_3,
	'NumOfHid_4' : R_NeuralNet.NumOfHid_4 ,
	'NumOfHid_5' : R_NeuralNet.NumOfHid_5 ,
	'NumOfHid_6' : R_NeuralNet.NumOfHid_6 ,
	'NumOfHid_7' : R_NeuralNet.NumOfHid_7 ,
	'NumOfHid_8' : R_NeuralNet.NumOfHid_8 ,
	'NumOfHid_9' : R_NeuralNet.NumOfHid_9 ,
	'NumOfStates' : R_NeuralNet.NumOfStates ,
	'OUTPUT' : OUTPUT }
	wieghts = {'wieghts_1' : R_NeuralNet.wieghts_1 ,
	'wieghts_2' : R_NeuralNet.wieghts_2 ,
	'wieghts_3' : R_NeuralNet.wieghts_3 ,
	'wieghts_4' : R_NeuralNet.wieghts_4 ,
	'wieghts_5' : R_NeuralNet.wieghts_5 ,
	'wieghts_6' : R_NeuralNet.wieghts_6 ,
	'wieghts_7' : R_NeuralNet.wieghts_7 ,
	'wieghts_8' : R_NeuralNet.wieghts_8 ,
	'wieghts_9' : R_NeuralNet.wieghts_9 ,
	'wieghts_10' : R_NeuralNet.wieghts_10 ,
	'wieghts_11' : R_NeuralNet.wieghts_11 ,
	'wieghts_12' : R_NeuralNet.wieghts_12 ,
	'wieghts_13' : R_NeuralNet.wieghts_13 ,
	'wieghts_14' : R_NeuralNet.wieghts_14 ,
	'wieghts_15' : R_NeuralNet.wieghts_15 ,
	'wieghts_16' : R_NeuralNet.wieghts_16 ,
	'wieghts_17' : R_NeuralNet.wieghts_17 ,
	'wieghts_18' : R_NeuralNet.wieghts_18 ,
	'wieghts_19' : R_NeuralNet.wieghts_19 ,
	'wieghts_20' : R_NeuralNet.wieghts_20 ,
	'wieghts_21' : R_NeuralNet.wieghts_21 ,
	'wieghts_22' : R_NeuralNet.wieghts_22 ,
	'wieghts_23' : R_NeuralNet.wieghts_23 ,
	'wieghts_24' : R_NeuralNet.wieghts_24 ,
	'wieghts_25' : R_NeuralNet.wieghts_25 ,
	'wieghts_26' : R_NeuralNet.wieghts_26 ,
	'wieghts_27' : R_NeuralNet.wieghts_27 ,
	'wieghts_28' : R_NeuralNet.wieghts_28 ,
	'wieghts_29' : R_NeuralNet.wieghts_29 ,
	'wieghts_30' : R_NeuralNet.wieghts_30 ,
	'wieghts_31' : R_NeuralNet.wieghts_31 ,
	'wieghts_32' : R_NeuralNet.wieghts_32 ,
	'wieghts_33' : R_NeuralNet.wieghts_33 ,
	'wieghts_34' : R_NeuralNet.wieghts_34 ,
	'wieghts_35' : R_NeuralNet.wieghts_35 ,
	'wieghts_36' : R_NeuralNet.wieghts_36 ,
	'wieghts_37' : R_NeuralNet.wieghts_37 ,
	'wieghts_38' : R_NeuralNet.wieghts_38 ,
	'wieghts_39' : R_NeuralNet.wieghts_39 ,
	'wieghts_40' : R_NeuralNet.wieghts_40 ,
	'wieghts_41' : R_NeuralNet.wieghts_41 ,
	'wieghts_42' : R_NeuralNet.wieghts_42 ,
	'wieghts_43' : R_NeuralNet.wieghts_43 ,
	'wieghts_44' : R_NeuralNet.wieghts_44 ,
	'wieghts_45' : R_NeuralNet.wieghts_45 ,
	'wieghts_46' : R_NeuralNet.wieghts_46 ,
	'wieghts_47' : R_NeuralNet.wieghts_47 ,
	'wieghts_48' : R_NeuralNet.wieghts_48 ,
	'wieghts_49' : R_NeuralNet.wieghts_49 ,
	'wieghts_50' : R_NeuralNet.wieghts_50 ,
	'wieghts_51' : R_NeuralNet.wieghts_51 ,
	'wieghts_52' : R_NeuralNet.wieghts_52 ,
	'wieghts_53' : R_NeuralNet.wieghts_53 ,
	'wieghts_54' : R_NeuralNet.wieghts_54 ,
	'wieghts_55' : R_NeuralNet.wieghts_55 ,
	'wieghts_s1' : R_NeuralNet.wieghts_s1 ,
	'wieghts_s2' : R_NeuralNet.wieghts_s2 ,
	'wieghts_s3' : R_NeuralNet.wieghts_s3 ,
	'wieghts_s4' : R_NeuralNet.wieghts_s4 ,
	'wieghts_s5' : R_NeuralNet.wieghts_s5 ,
	'wieghts_s6' : R_NeuralNet.wieghts_s6 ,
	'wieghts_s7' : R_NeuralNet.wieghts_s7 ,
	'wieghts_s8' : R_NeuralNet.wieghts_s8 ,
	'wieghts_s9' : R_NeuralNet.wieghts_s9 ,
	'wieghts_s10' : R_NeuralNet.wieghts_s10 ,
	'wieghts_s11' : R_NeuralNet.wieghts_s11 ,
	'wieghts_s12' : R_NeuralNet.wieghts_s12 ,
	'wieghts_s13' : R_NeuralNet.wieghts_s13 ,
	'wieghts_s14' : R_NeuralNet.wieghts_s14 ,
	'wieghts_s15' : R_NeuralNet.wieghts_s15 ,
	'wieghts_s16' : R_NeuralNet.wieghts_s16 ,
	'wieghts_s17' : R_NeuralNet.wieghts_s17 ,
	'wieghts_s18' : R_NeuralNet.wieghts_s18 ,
	'wieghts_s19' : R_NeuralNet.wieghts_s19 ,
	'wieghts_s20' : R_NeuralNet.wieghts_s20 ,
	'BaisIn' : R_NeuralNet.BaisIn ,
	'BaisOut' : R_NeuralNet.BaisOut ,
	'BaisHid_1' : R_NeuralNet.BaisHid_1 ,
	'BaisHid_2' : R_NeuralNet.BaisHid_2 ,
	'BaisHid_3' : R_NeuralNet.BaisHid_3 ,
	'BaisHid_4' : R_NeuralNet.BaisHid_4 ,
	'BaisHid_5' : R_NeuralNet.BaisHid_5 ,
	'BaisHid_6' : R_NeuralNet.BaisHid_6 ,
	'BaisHid_7' : R_NeuralNet.BaisHid_7 ,
	'BaisHid_8' : R_NeuralNet.BaisHid_8 ,
	'BaisHid_9' : R_NeuralNet.BaisHid_9 
	}
	np.save(os.path.join(path, 'Architecture.npy'),Architecture)
	np.save(os.path.join(path, 'wieghts.npy'),wieghts)
	return()

#Mutation funtion 
mutaion funtion that performs mutations of creatures.

In [ ]:
def mutate(R_NeuralNet):
	R_NeuralNet = R_NeuralNet.mutate()
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_1 = R_NeuralNet.NumOfHid_1 + 1
				new_hidden_w1 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws11 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w11 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w12 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w13 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w14 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w15 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w16 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w17 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w18 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w19 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws2 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_1 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_1-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_1 = np.insert(R_NeuralNet.wieghts_1,p,new_hidden_w1, axis = 0)
				R_NeuralNet.wieghts_s11 = np.insert(R_NeuralNet.wieghts_s11,p,new_hidden_ws11,axis = 0)
				R_NeuralNet.wieghts_11 = np.insert(R_NeuralNet.wieghts_11,p,new_hidden_w11, axis = 1)
				R_NeuralNet.wieghts_12 = np.insert(R_NeuralNet.wieghts_12,p,new_hidden_w12, axis = 1)
				R_NeuralNet.wieghts_13 = np.insert(R_NeuralNet.wieghts_13,p,new_hidden_w13, axis = 1)
				R_NeuralNet.wieghts_14 = np.insert(R_NeuralNet.wieghts_14,p,new_hidden_w14, axis = 1)
				R_NeuralNet.wieghts_15 = np.insert(R_NeuralNet.wieghts_15,p,new_hidden_w15, axis = 1)
				R_NeuralNet.wieghts_16 = np.insert(R_NeuralNet.wieghts_16,p,new_hidden_w16, axis = 1)
				R_NeuralNet.wieghts_17 = np.insert(R_NeuralNet.wieghts_17,p,new_hidden_w17, axis = 1)
				R_NeuralNet.wieghts_18 = np.insert(R_NeuralNet.wieghts_18,p,new_hidden_w18, axis = 1)
				R_NeuralNet.wieghts_19 = np.insert(R_NeuralNet.wieghts_19,p,new_hidden_w19, axis = 1)
				R_NeuralNet.wieghts_s2 = np.insert(R_NeuralNet.wieghts_s2,p,new_hidden_ws2,axis = 1)
				R_NeuralNet.BaisHid_1 = np.insert(R_NeuralNet.BaisHid_1,p,new_bais_hid_1,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_1 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_1-1)
				R_NeuralNet.wieghts_1 = np.delete(R_NeuralNet.wieghts_1,p,axis = 0)
				R_NeuralNet.wieghts_s11 = np.delete(R_NeuralNet.wieghts_s11,p,axis = 0)
				R_NeuralNet.wieghts_11 = np.delete(R_NeuralNet.wieghts_11,p, axis = 1)
				R_NeuralNet.wieghts_12 = np.delete(R_NeuralNet.wieghts_12,p, axis = 1)
				R_NeuralNet.wieghts_13 = np.delete(R_NeuralNet.wieghts_13,p, axis = 1)
				R_NeuralNet.wieghts_14 = np.delete(R_NeuralNet.wieghts_14,p, axis = 1)
				R_NeuralNet.wieghts_15 = np.delete(R_NeuralNet.wieghts_15,p, axis = 1)
				R_NeuralNet.wieghts_16 = np.delete(R_NeuralNet.wieghts_16,p, axis = 1)
				R_NeuralNet.wieghts_17 = np.delete(R_NeuralNet.wieghts_17,p, axis = 1)
				R_NeuralNet.wieghts_18 = np.delete(R_NeuralNet.wieghts_18,p, axis = 1)
				R_NeuralNet.wieghts_19 = np.delete(R_NeuralNet.wieghts_19,p, axis = 1)
				R_NeuralNet.wieghts_s2 = np.delete(R_NeuralNet.wieghts_s2,p,axis = 1)
				R_NeuralNet.BaisHid_1 = np.delete(R_NeuralNet.BaisHid_1,p,axis = 0)
				R_NeuralNet.NumOfHid_1 = R_NeuralNet.NumOfHid_1 - 1
				#delete a random hidden node
				#hidden_2
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_2 = R_NeuralNet.NumOfHid_2 + 1
				new_hidden_w2 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws12 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w11 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w20 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w21 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w22 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w23 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w24 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w25 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w26 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w27 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws3 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_2 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_2-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_2 = np.insert(R_NeuralNet.wieghts_2,p,new_hidden_w2, axis = 0)
				R_NeuralNet.wieghts_s12 = np.insert(R_NeuralNet.wieghts_s12,p,new_hidden_ws12,axis = 0)
				R_NeuralNet.wieghts_11 = np.insert(R_NeuralNet.wieghts_11,p,new_hidden_w11, axis = 0)
				R_NeuralNet.wieghts_20 = np.insert(R_NeuralNet.wieghts_20,p,new_hidden_w20, axis = 1)
				R_NeuralNet.wieghts_21 = np.insert(R_NeuralNet.wieghts_21,p,new_hidden_w21, axis = 1)
				R_NeuralNet.wieghts_22 = np.insert(R_NeuralNet.wieghts_22,p,new_hidden_w22, axis = 1)
				R_NeuralNet.wieghts_23 = np.insert(R_NeuralNet.wieghts_23,p,new_hidden_w23, axis = 1)
				R_NeuralNet.wieghts_24 = np.insert(R_NeuralNet.wieghts_24,p,new_hidden_w24, axis = 1)
				R_NeuralNet.wieghts_25 = np.insert(R_NeuralNet.wieghts_25,p,new_hidden_w25, axis = 1)
				R_NeuralNet.wieghts_26 = np.insert(R_NeuralNet.wieghts_26,p,new_hidden_w26, axis = 1)
				R_NeuralNet.wieghts_27 = np.insert(R_NeuralNet.wieghts_27,p,new_hidden_w27, axis = 1)
				R_NeuralNet.wieghts_s3 = np.insert(R_NeuralNet.wieghts_s3,p,new_hidden_ws3,axis = 1)
				R_NeuralNet.BaisHid_2 = np.insert(R_NeuralNet.BaisHid_2,p,new_bais_hid_2,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_2 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_2-1)
				R_NeuralNet.wieghts_2 = np.delete(R_NeuralNet.wieghts_2,p,axis = 0)
				R_NeuralNet.wieghts_s12 = np.delete(R_NeuralNet.wieghts_s12,p,axis = 0)
				R_NeuralNet.wieghts_11 = np.delete(R_NeuralNet.wieghts_11,p, axis = 0)
				R_NeuralNet.wieghts_20 = np.delete(R_NeuralNet.wieghts_20,p, axis = 1)
				R_NeuralNet.wieghts_21 = np.delete(R_NeuralNet.wieghts_21,p, axis = 1)
				R_NeuralNet.wieghts_22 = np.delete(R_NeuralNet.wieghts_22,p, axis = 1)
				R_NeuralNet.wieghts_23 = np.delete(R_NeuralNet.wieghts_23,p, axis = 1)
				R_NeuralNet.wieghts_24 = np.delete(R_NeuralNet.wieghts_24,p, axis = 1)
				R_NeuralNet.wieghts_25 = np.delete(R_NeuralNet.wieghts_25,p, axis = 1)
				R_NeuralNet.wieghts_26 = np.delete(R_NeuralNet.wieghts_26,p, axis = 1)
				R_NeuralNet.wieghts_27 = np.delete(R_NeuralNet.wieghts_27,p, axis = 1)
				R_NeuralNet.wieghts_s3 = np.delete(R_NeuralNet.wieghts_s3,p,axis = 1)
				R_NeuralNet.BaisHid_2 = np.delete(R_NeuralNet.BaisHid_2,p,axis = 0)
				R_NeuralNet.NumOfHid_2 = R_NeuralNet.NumOfHid_2 - 1
				#delete a random hidden node
				#hidden_3
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_3 = R_NeuralNet.NumOfHid_3 + 1
				new_hidden_w3 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws13 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w12 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w20 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w28 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w29 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w30 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w31 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w32 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w33 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w34 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws4 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_3 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_3-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_3 = np.insert(R_NeuralNet.wieghts_3,p,new_hidden_w3, axis = 0)
				R_NeuralNet.wieghts_s13 = np.insert(R_NeuralNet.wieghts_s13,p,new_hidden_ws13,axis = 0)
				R_NeuralNet.wieghts_12 = np.insert(R_NeuralNet.wieghts_12,p,new_hidden_w12, axis = 0)
				R_NeuralNet.wieghts_20 = np.insert(R_NeuralNet.wieghts_20,p,new_hidden_w20, axis = 0)
				R_NeuralNet.wieghts_28 = np.insert(R_NeuralNet.wieghts_28,p,new_hidden_w28, axis = 1)
				R_NeuralNet.wieghts_29 = np.insert(R_NeuralNet.wieghts_29,p,new_hidden_w29, axis = 1)
				R_NeuralNet.wieghts_30 = np.insert(R_NeuralNet.wieghts_30,p,new_hidden_w30, axis = 1)
				R_NeuralNet.wieghts_31 = np.insert(R_NeuralNet.wieghts_31,p,new_hidden_w31, axis = 1)
				R_NeuralNet.wieghts_32 = np.insert(R_NeuralNet.wieghts_32,p,new_hidden_w32, axis = 1)
				R_NeuralNet.wieghts_33 = np.insert(R_NeuralNet.wieghts_33,p,new_hidden_w33, axis = 1)
				R_NeuralNet.wieghts_34 = np.insert(R_NeuralNet.wieghts_34,p,new_hidden_w34, axis = 1)
				R_NeuralNet.wieghts_s4 = np.insert(R_NeuralNet.wieghts_s4,p,new_hidden_ws4,axis = 1)
				R_NeuralNet.BaisHid_3 = np.insert(R_NeuralNet.BaisHid_3,p,new_bais_hid_3,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_3 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_3-1)
				R_NeuralNet.wieghts_3 = np.delete(R_NeuralNet.wieghts_3,p,axis = 0)
				R_NeuralNet.wieghts_s13 = np.delete(R_NeuralNet.wieghts_s13,p,axis = 0)
				R_NeuralNet.wieghts_12 = np.delete(R_NeuralNet.wieghts_12,p, axis = 0)
				R_NeuralNet.wieghts_20 = np.delete(R_NeuralNet.wieghts_20,p, axis = 0)
				R_NeuralNet.wieghts_28 = np.delete(R_NeuralNet.wieghts_28,p, axis = 1)
				R_NeuralNet.wieghts_29 = np.delete(R_NeuralNet.wieghts_29,p, axis = 1)
				R_NeuralNet.wieghts_30 = np.delete(R_NeuralNet.wieghts_30,p, axis = 1)
				R_NeuralNet.wieghts_31 = np.delete(R_NeuralNet.wieghts_31,p, axis = 1)
				R_NeuralNet.wieghts_32 = np.delete(R_NeuralNet.wieghts_32,p, axis = 1)
				R_NeuralNet.wieghts_33 = np.delete(R_NeuralNet.wieghts_33,p, axis = 1)
				R_NeuralNet.wieghts_34 = np.delete(R_NeuralNet.wieghts_34,p, axis = 1)
				R_NeuralNet.wieghts_s4 = np.delete(R_NeuralNet.wieghts_s4,p,axis = 1)
				R_NeuralNet.BaisHid_3 = np.delete(R_NeuralNet.BaisHid_3,p,axis = 0)
				R_NeuralNet.NumOfHid_3 = R_NeuralNet.NumOfHid_3 - 1
				#delete a random hidden node
				#hidden_4
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_4 = R_NeuralNet.NumOfHid_4 + 1
				new_hidden_w4 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws14 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w13 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w21 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w28 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w35 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w36 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w37 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w38 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w39 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w40 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws5 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_4 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_4-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_4 = np.insert(R_NeuralNet.wieghts_4,p,new_hidden_w4, axis = 0)
				R_NeuralNet.wieghts_s14 = np.insert(R_NeuralNet.wieghts_s14,p,new_hidden_ws14,axis = 0)
				R_NeuralNet.wieghts_13 = np.insert(R_NeuralNet.wieghts_13,p,new_hidden_w13, axis = 0)
				R_NeuralNet.wieghts_21 = np.insert(R_NeuralNet.wieghts_21,p,new_hidden_w21, axis = 0)
				R_NeuralNet.wieghts_28 = np.insert(R_NeuralNet.wieghts_28,p,new_hidden_w28, axis = 0)
				R_NeuralNet.wieghts_35 = np.insert(R_NeuralNet.wieghts_35,p,new_hidden_w35, axis = 1)
				R_NeuralNet.wieghts_36 = np.insert(R_NeuralNet.wieghts_36,p,new_hidden_w36, axis = 1)
				R_NeuralNet.wieghts_37 = np.insert(R_NeuralNet.wieghts_37,p,new_hidden_w37, axis = 1)
				R_NeuralNet.wieghts_38 = np.insert(R_NeuralNet.wieghts_38,p,new_hidden_w38, axis = 1)
				R_NeuralNet.wieghts_39 = np.insert(R_NeuralNet.wieghts_39,p,new_hidden_w39, axis = 1)
				R_NeuralNet.wieghts_40 = np.insert(R_NeuralNet.wieghts_40,p,new_hidden_w40, axis = 1)
				R_NeuralNet.wieghts_s5 = np.insert(R_NeuralNet.wieghts_s5,p,new_hidden_ws5,axis = 1)
				R_NeuralNet.BaisHid_4 = np.insert(R_NeuralNet.BaisHid_4,p,new_bais_hid_4,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_4 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_4-1)
				R_NeuralNet.wieghts_4 = np.delete(R_NeuralNet.wieghts_4,p,axis = 0)
				R_NeuralNet.wieghts_s14 = np.delete(R_NeuralNet.wieghts_s14,p,axis = 0)
				R_NeuralNet.wieghts_13 = np.delete(R_NeuralNet.wieghts_13,p, axis = 0)
				R_NeuralNet.wieghts_21 = np.delete(R_NeuralNet.wieghts_21,p, axis = 0)
				R_NeuralNet.wieghts_28 = np.delete(R_NeuralNet.wieghts_28,p, axis = 0)
				R_NeuralNet.wieghts_35 = np.delete(R_NeuralNet.wieghts_35,p, axis = 1)
				R_NeuralNet.wieghts_36 = np.delete(R_NeuralNet.wieghts_36,p, axis = 1)
				R_NeuralNet.wieghts_37 = np.delete(R_NeuralNet.wieghts_37,p, axis = 1)
				R_NeuralNet.wieghts_38 = np.delete(R_NeuralNet.wieghts_38,p, axis = 1)
				R_NeuralNet.wieghts_39 = np.delete(R_NeuralNet.wieghts_39,p, axis = 1)
				R_NeuralNet.wieghts_40 = np.delete(R_NeuralNet.wieghts_40,p, axis = 1)
				R_NeuralNet.wieghts_s5 = np.delete(R_NeuralNet.wieghts_s5,p,axis = 1)
				R_NeuralNet.BaisHid_4 = np.delete(R_NeuralNet.BaisHid_4,p,axis = 0)
				R_NeuralNet.NumOfHid_4 = R_NeuralNet.NumOfHid_4 - 1
				#delete a random hidden node
				#hidden_5
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_5 = R_NeuralNet.NumOfHid_5 + 1
				new_hidden_w5 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws15 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w14 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w22 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w29 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w35 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w41 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w42 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w43 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w44 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w45 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws6 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_5 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_5-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_5 = np.insert(R_NeuralNet.wieghts_5,p,new_hidden_w5, axis = 0)
				R_NeuralNet.wieghts_s15 = np.insert(R_NeuralNet.wieghts_s15,p,new_hidden_ws15,axis = 0)
				R_NeuralNet.wieghts_14 = np.insert(R_NeuralNet.wieghts_14,p,new_hidden_w14, axis = 0)
				R_NeuralNet.wieghts_22 = np.insert(R_NeuralNet.wieghts_22,p,new_hidden_w22, axis = 0)
				R_NeuralNet.wieghts_29 = np.insert(R_NeuralNet.wieghts_29,p,new_hidden_w29, axis = 0)
				R_NeuralNet.wieghts_35 = np.insert(R_NeuralNet.wieghts_35,p,new_hidden_w35, axis = 0)
				R_NeuralNet.wieghts_41 = np.insert(R_NeuralNet.wieghts_41,p,new_hidden_w41, axis = 1)
				R_NeuralNet.wieghts_42 = np.insert(R_NeuralNet.wieghts_42,p,new_hidden_w42, axis = 1)
				R_NeuralNet.wieghts_43 = np.insert(R_NeuralNet.wieghts_43,p,new_hidden_w43, axis = 1)
				R_NeuralNet.wieghts_44 = np.insert(R_NeuralNet.wieghts_44,p,new_hidden_w44, axis = 1)
				R_NeuralNet.wieghts_45 = np.insert(R_NeuralNet.wieghts_45,p,new_hidden_w45, axis = 1)
				R_NeuralNet.wieghts_s6 = np.insert(R_NeuralNet.wieghts_s6,p,new_hidden_ws6,axis = 1)
				R_NeuralNet.BaisHid_5 = np.insert(R_NeuralNet.BaisHid_5,p,new_bais_hid_5,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_5 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_5-1)
				R_NeuralNet.wieghts_5 = np.delete(R_NeuralNet.wieghts_5,p,axis = 0)
				R_NeuralNet.wieghts_s15 = np.delete(R_NeuralNet.wieghts_s15,p,axis = 0)
				R_NeuralNet.wieghts_14 = np.delete(R_NeuralNet.wieghts_14,p, axis = 0)
				R_NeuralNet.wieghts_22 = np.delete(R_NeuralNet.wieghts_22,p, axis = 0)
				R_NeuralNet.wieghts_29 = np.delete(R_NeuralNet.wieghts_29,p, axis = 0)
				R_NeuralNet.wieghts_35 = np.delete(R_NeuralNet.wieghts_35,p, axis = 0)
				R_NeuralNet.wieghts_41 = np.delete(R_NeuralNet.wieghts_41,p, axis = 1)
				R_NeuralNet.wieghts_42 = np.delete(R_NeuralNet.wieghts_42,p, axis = 1)
				R_NeuralNet.wieghts_43 = np.delete(R_NeuralNet.wieghts_43,p, axis = 1)
				R_NeuralNet.wieghts_44 = np.delete(R_NeuralNet.wieghts_44,p, axis = 1)
				R_NeuralNet.wieghts_45 = np.delete(R_NeuralNet.wieghts_45,p, axis = 1)
				R_NeuralNet.wieghts_s6 = np.delete(R_NeuralNet.wieghts_s6,p,axis = 1)
				R_NeuralNet.BaisHid_5 = np.delete(R_NeuralNet.BaisHid_5,p,axis = 0)
				R_NeuralNet.NumOfHid_5 = R_NeuralNet.NumOfHid_5 - 1
				#delete a random hidden node
				#hidden_6
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_6 = R_NeuralNet.NumOfHid_6 + 1
				new_hidden_w6 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws16 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w15 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w23 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w30 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w36 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w41 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w46 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w47 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w48 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w49 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws7 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_6 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_6-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_6 = np.insert(R_NeuralNet.wieghts_6,p,new_hidden_w6, axis = 0)
				R_NeuralNet.wieghts_s16 = np.insert(R_NeuralNet.wieghts_s16,p,new_hidden_ws16,axis = 0)
				R_NeuralNet.wieghts_15 = np.insert(R_NeuralNet.wieghts_15,p,new_hidden_w15, axis = 0)
				R_NeuralNet.wieghts_23 = np.insert(R_NeuralNet.wieghts_23,p,new_hidden_w23, axis = 0)
				R_NeuralNet.wieghts_30 = np.insert(R_NeuralNet.wieghts_30,p,new_hidden_w30, axis = 0)
				R_NeuralNet.wieghts_36 = np.insert(R_NeuralNet.wieghts_36,p,new_hidden_w36, axis = 0)
				R_NeuralNet.wieghts_41 = np.insert(R_NeuralNet.wieghts_41,p,new_hidden_w41, axis = 0)
				R_NeuralNet.wieghts_46 = np.insert(R_NeuralNet.wieghts_46,p,new_hidden_w46, axis = 1)
				R_NeuralNet.wieghts_47 = np.insert(R_NeuralNet.wieghts_47,p,new_hidden_w47, axis = 1)
				R_NeuralNet.wieghts_48 = np.insert(R_NeuralNet.wieghts_48,p,new_hidden_w48, axis = 1)
				R_NeuralNet.wieghts_49 = np.insert(R_NeuralNet.wieghts_49,p,new_hidden_w49, axis = 1)
				R_NeuralNet.wieghts_s7 = np.insert(R_NeuralNet.wieghts_s7,p,new_hidden_ws7,axis = 1)
				R_NeuralNet.BaisHid_6 = np.insert(R_NeuralNet.BaisHid_6,p,new_bais_hid_6,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_6 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_6-1)
				R_NeuralNet.wieghts_6 = np.delete(R_NeuralNet.wieghts_6,p,axis = 0)
				R_NeuralNet.wieghts_s16 = np.delete(R_NeuralNet.wieghts_s16,p,axis = 0)
				R_NeuralNet.wieghts_15 = np.delete(R_NeuralNet.wieghts_15,p, axis = 0)
				R_NeuralNet.wieghts_23 = np.delete(R_NeuralNet.wieghts_23,p, axis = 0)
				R_NeuralNet.wieghts_30 = np.delete(R_NeuralNet.wieghts_30,p, axis = 0)
				R_NeuralNet.wieghts_36 = np.delete(R_NeuralNet.wieghts_36,p, axis = 0)
				R_NeuralNet.wieghts_41 = np.delete(R_NeuralNet.wieghts_41,p, axis = 0)
				R_NeuralNet.wieghts_46 = np.delete(R_NeuralNet.wieghts_46,p, axis = 1)
				R_NeuralNet.wieghts_47 = np.delete(R_NeuralNet.wieghts_47,p, axis = 1)
				R_NeuralNet.wieghts_48 = np.delete(R_NeuralNet.wieghts_48,p, axis = 1)
				R_NeuralNet.wieghts_49 = np.delete(R_NeuralNet.wieghts_49,p, axis = 1)
				R_NeuralNet.wieghts_s7 = np.delete(R_NeuralNet.wieghts_s7,p,axis = 1)
				R_NeuralNet.BaisHid_6 = np.delete(R_NeuralNet.BaisHid_6,p,axis = 0)
				R_NeuralNet.NumOfHid_6 = R_NeuralNet.NumOfHid_6 - 1
				#delete a random hidden node
				#hidden_7
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_7 = R_NeuralNet.NumOfHid_7 + 1
				new_hidden_w7 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws17 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w16 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w24 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w31 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w37 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w42 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w46 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w50 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w51 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w52 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws8 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_7 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_7-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_7 = np.insert(R_NeuralNet.wieghts_7,p,new_hidden_w7, axis = 0)
				R_NeuralNet.wieghts_s17 = np.insert(R_NeuralNet.wieghts_s17,p,new_hidden_ws17,axis =0)
				R_NeuralNet.wieghts_16 = np.insert(R_NeuralNet.wieghts_16,p,new_hidden_w16, axis = 0)
				R_NeuralNet.wieghts_24 = np.insert(R_NeuralNet.wieghts_24,p,new_hidden_w24, axis = 0)
				R_NeuralNet.wieghts_31 = np.insert(R_NeuralNet.wieghts_31,p,new_hidden_w31, axis = 0)
				R_NeuralNet.wieghts_37 = np.insert(R_NeuralNet.wieghts_37,p,new_hidden_w37, axis = 0)
				R_NeuralNet.wieghts_42 = np.insert(R_NeuralNet.wieghts_42,p,new_hidden_w42, axis = 0)
				R_NeuralNet.wieghts_46 = np.insert(R_NeuralNet.wieghts_46,p,new_hidden_w46, axis = 0)
				R_NeuralNet.wieghts_50 = np.insert(R_NeuralNet.wieghts_50,p,new_hidden_w50, axis = 1)
				R_NeuralNet.wieghts_51 = np.insert(R_NeuralNet.wieghts_51,p,new_hidden_w51, axis = 1)
				R_NeuralNet.wieghts_52 = np.insert(R_NeuralNet.wieghts_52,p,new_hidden_w52, axis = 1)
				R_NeuralNet.wieghts_s8 = np.insert(R_NeuralNet.wieghts_s8,p,new_hidden_ws8,axis = 1)
				R_NeuralNet.BaisHid_7 = np.insert(R_NeuralNet.BaisHid_7,p,new_bais_hid_7,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_7 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_7-1)
				R_NeuralNet.wieghts_7 = np.delete(R_NeuralNet.wieghts_7,p,axis = 0)
				R_NeuralNet.wieghts_s17 = np.delete(R_NeuralNet.wieghts_s17,p,axis = 0)
				R_NeuralNet.wieghts_16 = np.delete(R_NeuralNet.wieghts_16,p, axis = 0)
				R_NeuralNet.wieghts_24 = np.delete(R_NeuralNet.wieghts_24,p, axis = 0)
				R_NeuralNet.wieghts_31 = np.delete(R_NeuralNet.wieghts_31,p, axis = 0)
				R_NeuralNet.wieghts_37 = np.delete(R_NeuralNet.wieghts_37,p, axis = 0)
				R_NeuralNet.wieghts_42 = np.delete(R_NeuralNet.wieghts_42,p, axis = 0)
				R_NeuralNet.wieghts_46 = np.delete(R_NeuralNet.wieghts_46,p, axis = 0)
				R_NeuralNet.wieghts_50 = np.delete(R_NeuralNet.wieghts_50,p, axis = 1)
				R_NeuralNet.wieghts_51 = np.delete(R_NeuralNet.wieghts_51,p, axis = 1)
				R_NeuralNet.wieghts_52 = np.delete(R_NeuralNet.wieghts_52,p, axis = 1)
				R_NeuralNet.wieghts_s8 = np.delete(R_NeuralNet.wieghts_s8,p,axis = 1)
				R_NeuralNet.BaisHid_7 = np.delete(R_NeuralNet.BaisHid_7,p,axis = 0)
				R_NeuralNet.NumOfHid_7 = R_NeuralNet.NumOfHid_7 - 1
				#delete a random hidden node
				#hidden_8
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_8 = R_NeuralNet.NumOfHid_8 + 1
				new_hidden_w8 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws18 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w17 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w25 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w32 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w38 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w43 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w47 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w50 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w53 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_w54 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws9 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_bais_hid_8 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_8-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_8 = np.insert(R_NeuralNet.wieghts_8,p,new_hidden_w8, axis = 0)
				R_NeuralNet.wieghts_s18 = np.insert(R_NeuralNet.wieghts_s18,p,new_hidden_ws18,axis = 0)
				R_NeuralNet.wieghts_17 = np.insert(R_NeuralNet.wieghts_17,p,new_hidden_w17, axis = 0)
				R_NeuralNet.wieghts_25 = np.insert(R_NeuralNet.wieghts_25,p,new_hidden_w25, axis = 0)
				R_NeuralNet.wieghts_32 = np.insert(R_NeuralNet.wieghts_32,p,new_hidden_w32, axis = 0)
				R_NeuralNet.wieghts_38 = np.insert(R_NeuralNet.wieghts_38,p,new_hidden_w38, axis = 0)
				R_NeuralNet.wieghts_43 = np.insert(R_NeuralNet.wieghts_43,p,new_hidden_w43, axis = 0)
				R_NeuralNet.wieghts_47 = np.insert(R_NeuralNet.wieghts_47,p,new_hidden_w47, axis = 0)
				R_NeuralNet.wieghts_50 = np.insert(R_NeuralNet.wieghts_50,p,new_hidden_w50, axis = 0)
				R_NeuralNet.wieghts_53 = np.insert(R_NeuralNet.wieghts_53,p,new_hidden_w53, axis = 1)
				R_NeuralNet.wieghts_54 = np.insert(R_NeuralNet.wieghts_54,p,new_hidden_w54, axis = 1)
				R_NeuralNet.wieghts_s9 = np.insert(R_NeuralNet.wieghts_s9,p,new_hidden_ws9,axis = 1)
				R_NeuralNet.BaisHid_8 = np.insert(R_NeuralNet.BaisHid_8,p,new_bais_hid_8,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_8 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_8-1)
				R_NeuralNet.wieghts_8 = np.delete(R_NeuralNet.wieghts_8,p,axis = 0)
				R_NeuralNet.wieghts_s18 = np.delete(R_NeuralNet.wieghts_s18,p,axis = 0)
				R_NeuralNet.wieghts_17 = np.delete(R_NeuralNet.wieghts_17,p, axis = 0)
				R_NeuralNet.wieghts_25 = np.delete(R_NeuralNet.wieghts_25,p, axis = 0)
				R_NeuralNet.wieghts_32 = np.delete(R_NeuralNet.wieghts_32,p, axis = 0)
				R_NeuralNet.wieghts_38 = np.delete(R_NeuralNet.wieghts_38,p, axis = 0)
				R_NeuralNet.wieghts_43 = np.delete(R_NeuralNet.wieghts_43,p, axis = 0)
				R_NeuralNet.wieghts_47 = np.delete(R_NeuralNet.wieghts_47,p, axis = 0)
				R_NeuralNet.wieghts_50 = np.delete(R_NeuralNet.wieghts_50,p, axis = 0)
				R_NeuralNet.wieghts_53 = np.delete(R_NeuralNet.wieghts_53,p, axis = 1)
				R_NeuralNet.wieghts_54 = np.delete(R_NeuralNet.wieghts_54,p, axis = 1)
				R_NeuralNet.wieghts_s9 = np.delete(R_NeuralNet.wieghts_s9,p,axis = 1)
				R_NeuralNet.BaisHid_8 = np.delete(R_NeuralNet.BaisHid_8,p,axis = 0)
				R_NeuralNet.NumOfHid_8 = R_NeuralNet.NumOfHid_8 - 1
				#delete a random hidden node
				#hidden_9
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfHid_9 = R_NeuralNet.NumOfHid_9 + 1
				new_hidden_w9 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws19 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfStates))
				new_hidden_w18 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_w26 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_w33 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_w39 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_w44 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_w48 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_w51 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_w53 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_w55 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				new_hidden_ws10 = np.random.uniform(R_NeuralNet.NumOfStates)
				new_bais_hid_9 = np.random.uniform(INITIAL_BMIN,INITIAL_BMAX,(1,1))
				p = random.randint(0,R_NeuralNet.NumOfHid_9-1)
				#add a new a random hidden node
				R_NeuralNet.wieghts_9 = np.insert(R_NeuralNet.wieghts_9,p,new_hidden_w9, axis = 0)
				R_NeuralNet.wieghts_s19 = np.insert(R_NeuralNet.wieghts_s19,p,new_hidden_ws19,axis = 0)
				R_NeuralNet.wieghts_18 = np.insert(R_NeuralNet.wieghts_18,p,new_hidden_w18, axis = 0)
				R_NeuralNet.wieghts_26 = np.insert(R_NeuralNet.wieghts_26,p,new_hidden_w26, axis = 0)
				R_NeuralNet.wieghts_33 = np.insert(R_NeuralNet.wieghts_33,p,new_hidden_w33, axis = 0)
				R_NeuralNet.wieghts_39 = np.insert(R_NeuralNet.wieghts_39,p,new_hidden_w39, axis = 0)
				R_NeuralNet.wieghts_44 = np.insert(R_NeuralNet.wieghts_44,p,new_hidden_w44, axis = 0)
				R_NeuralNet.wieghts_48 = np.insert(R_NeuralNet.wieghts_48,p,new_hidden_w48, axis = 0)
				R_NeuralNet.wieghts_51 = np.insert(R_NeuralNet.wieghts_51,p,new_hidden_w51, axis = 0)
				R_NeuralNet.wieghts_53 = np.insert(R_NeuralNet.wieghts_53,p,new_hidden_w53, axis = 0)
				R_NeuralNet.wieghts_55 = np.insert(R_NeuralNet.wieghts_55,p,new_hidden_w55, axis = 1)
				R_NeuralNet.wieghts_s10 = np.insert(R_NeuralNet.wieghts_s10,p,new_hidden_ws10,axis = 1)
				R_NeuralNet.BaisHid_9 = np.insert(R_NeuralNet.BaisHid_9,p,new_bais_hid_9,axis = 0)
		else:
			if(R_NeuralNet.NumOfHid_9 > 0):
				p = random.randint(0,R_NeuralNet.NumOfHid_9-1)
				R_NeuralNet.wieghts_9 = np.delete(R_NeuralNet.wieghts_9,p,axis = 0)
				R_NeuralNet.wieghts_s19 = np.delete(R_NeuralNet.wieghts_s19,p,axis = 0)
				R_NeuralNet.wieghts_18 = np.delete(R_NeuralNet.wieghts_18,p, axis = 0)
				R_NeuralNet.wieghts_26 = np.delete(R_NeuralNet.wieghts_26,p, axis = 0)
				R_NeuralNet.wieghts_33 = np.delete(R_NeuralNet.wieghts_33,p, axis = 0)
				R_NeuralNet.wieghts_39 = np.delete(R_NeuralNet.wieghts_39,p, axis = 0)
				R_NeuralNet.wieghts_44 = np.delete(R_NeuralNet.wieghts_44,p, axis = 0)
				R_NeuralNet.wieghts_48 = np.delete(R_NeuralNet.wieghts_48,p, axis = 0)
				R_NeuralNet.wieghts_51 = np.delete(R_NeuralNet.wieghts_51,p, axis = 0)
				R_NeuralNet.wieghts_53 = np.delete(R_NeuralNet.wieghts_53,p, axis = 0)
				R_NeuralNet.wieghts_55 = np.delete(R_NeuralNet.wieghts_55,p, axis = 1)
				R_NeuralNet.wieghts_s10 = np.delete(R_NeuralNet.wieghts_s10,p,axis = 1)
				R_NeuralNet.BaisHid_9 = np.delete(R_NeuralNet.BaisHid_9,p,axis = 0)
				R_NeuralNet.NumOfHid_9 = R_NeuralNet.NumOfHid_9 - 1
				#delete a random hidden node
				# states mutation
	l = random.randint(0,100)
	if l <= int(MUTATION_RATE*100) :
		s = random.randint(0,1)
		if s == 0 :
				R_NeuralNet.NumOfStates = R_NeuralNet.NumOfStates + 1
				new_hidden_ws1 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(INPUT))
				new_hidden_ws2 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_ws3 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_ws4 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_ws5 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_ws6 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_ws7 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_ws8 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_ws9 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_ws10 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_ws11 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_1))
				new_hidden_ws12 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_2))
				new_hidden_ws13 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_3))
				new_hidden_ws14 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_4))
				new_hidden_ws15 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_5))
				new_hidden_ws16 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_6))
				new_hidden_ws17 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_7))
				new_hidden_ws18 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_8))
				new_hidden_ws19 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(R_NeuralNet.NumOfHid_9))
				new_hidden_ws20 = np.random.uniform(INITIAL_WMIN,INITIAL_WMAX,(OUTPUT))
				p = random.randint(0,R_NeuralNet.NumOfStates-1)
				#add a new a state variable with random wieghtes node
				R_NeuralNet.wieghts_s1 = np.insert(R_NeuralNet.wieghts_s1,p,new_hidden_ws1,axis = 0)
				R_NeuralNet.wieghts_s2 = np.insert(R_NeuralNet.wieghts_s2,p,new_hidden_ws2,axis = 0)
				R_NeuralNet.wieghts_s3 = np.insert(R_NeuralNet.wieghts_s3,p,new_hidden_ws3,axis = 0)
				R_NeuralNet.wieghts_s4 = np.insert(R_NeuralNet.wieghts_s4,p,new_hidden_ws4,axis = 0)
				R_NeuralNet.wieghts_s5 = np.insert(R_NeuralNet.wieghts_s5,p,new_hidden_ws5,axis = 0)
				R_NeuralNet.wieghts_s6 = np.insert(R_NeuralNet.wieghts_s6,p,new_hidden_ws6,axis = 0)
				R_NeuralNet.wieghts_s7 = np.insert(R_NeuralNet.wieghts_s7,p,new_hidden_ws7,axis = 0)
				R_NeuralNet.wieghts_s8 = np.insert(R_NeuralNet.wieghts_s8,p,new_hidden_ws8,axis = 0)
				R_NeuralNet.wieghts_s9 = np.insert(R_NeuralNet.wieghts_s9,p,new_hidden_ws9,axis = 0)
				R_NeuralNet.wieghts_s10 = np.insert(R_NeuralNet.wieghts_s10,p,new_hidden_ws10,axis = 0)
				R_NeuralNet.wieghts_s11 = np.insert(R_NeuralNet.wieghts_s11,p,new_hidden_ws11,axis = 1)
				R_NeuralNet.wieghts_s12 = np.insert(R_NeuralNet.wieghts_s12,p,new_hidden_ws12,axis = 1)
				R_NeuralNet.wieghts_s13 = np.insert(R_NeuralNet.wieghts_s13,p,new_hidden_ws13,axis = 1)
				R_NeuralNet.wieghts_s14 = np.insert(R_NeuralNet.wieghts_s14,p,new_hidden_ws14,axis = 1)
				R_NeuralNet.wieghts_s15 = np.insert(R_NeuralNet.wieghts_s15,p,new_hidden_ws15,axis = 1)
				R_NeuralNet.wieghts_s16 = np.insert(R_NeuralNet.wieghts_s16,p,new_hidden_ws16,axis = 1)
				R_NeuralNet.wieghts_s17 = np.insert(R_NeuralNet.wieghts_s17,p,new_hidden_ws17,axis = 1)
				R_NeuralNet.wieghts_s18 = np.insert(R_NeuralNet.wieghts_s18,p,new_hidden_ws18,axis = 1)
				R_NeuralNet.wieghts_s19 = np.insert(R_NeuralNet.wieghts_s19,p,new_hidden_ws19,axis = 1)
				R_NeuralNet.wieghts_s20 = np.insert(R_NeuralNet.wieghts_s20,p,new_hidden_ws20,axis = 1)
		else:
				#delete a state variable
			if(R_NeuralNet.NumOfStates > STATE_MIN):
				p = random.randint(0,R_NeuralNet.NumOfStates -1)
				R_NeuralNet.wieghts_s1 = np.delete(R_NeuralNet.wieghts_s1,p,axis = 0)
				R_NeuralNet.wieghts_s2 = np.delete(R_NeuralNet.wieghts_s2,p,axis = 0)
				R_NeuralNet.wieghts_s3 = np.delete(R_NeuralNet.wieghts_s3,p,axis = 0)
				R_NeuralNet.wieghts_s4 = np.delete(R_NeuralNet.wieghts_s4,p,axis = 0)
				R_NeuralNet.wieghts_s5 = np.delete(R_NeuralNet.wieghts_s5,p,axis = 0)
				R_NeuralNet.wieghts_s6 = np.delete(R_NeuralNet.wieghts_s6,p,axis = 0)
				R_NeuralNet.wieghts_s7 = np.delete(R_NeuralNet.wieghts_s7,p,axis = 0)
				R_NeuralNet.wieghts_s8 = np.delete(R_NeuralNet.wieghts_s8,p,axis = 0)
				R_NeuralNet.wieghts_s9 = np.delete(R_NeuralNet.wieghts_s9,p,axis = 0)
				R_NeuralNet.wieghts_s10 = np.delete(R_NeuralNet.wieghts_s10,p,axis = 0)
				R_NeuralNet.wieghts_s11 = np.delete(R_NeuralNet.wieghts_s11,p,axis = 1)
				R_NeuralNet.wieghts_s12 = np.delete(R_NeuralNet.wieghts_s12,p,axis = 1)
				R_NeuralNet.wieghts_s13 = np.delete(R_NeuralNet.wieghts_s13,p,axis = 1)
				R_NeuralNet.wieghts_s14 = np.delete(R_NeuralNet.wieghts_s14,p,axis = 1)
				R_NeuralNet.wieghts_s15 = np.delete(R_NeuralNet.wieghts_s15,p,axis = 1)
				R_NeuralNet.wieghts_s16 = np.delete(R_NeuralNet.wieghts_s16,p,axis = 1)
				R_NeuralNet.wieghts_s17 = np.delete(R_NeuralNet.wieghts_s17,p,axis = 1)
				R_NeuralNet.wieghts_s18 = np.delete(R_NeuralNet.wieghts_s18,p,axis = 1)
				R_NeuralNet.wieghts_s19 = np.delete(R_NeuralNet.wieghts_s19,p,axis = 1)
				R_NeuralNet.wieghts_s20 = np.delete(R_NeuralNet.wieghts_s20,p,axis = 1)
				R_NeuralNet.NumOfStates = R_NeuralNet.NumOfStates - 1
				R_NeuralNet.reset()
	return (R_NeuralNet)


# load funcion
loads the saved creature

In [ ]:
def load(R_NeuralNet , URL =''):
	try :
		Architecture = np.load(os.path.join(URL + path , 'Architecture.npy') ,allow_pickle=True).item()
		wieghts  = np.load(os.path.join(URL + path ,'wieghts.npy'),allow_pickle=True).item()
		if(Architecture['INPUT'] != INPUT or Architecture['OUTPUT'] != OUTPUT ):
			print("the creature cannot be loaded due to INPUT OUTPUT missmatch !!!!")
			return()
		else :
			R_NeuralNet.NumOfHid_1 = Architecture['NumOfHid_1']
			R_NeuralNet.NumOfHid_2 = Architecture['NumOfHid_2']
			R_NeuralNet.NumOfHid_3 = Architecture['NumOfHid_3']
			R_NeuralNet.NumOfHid_4 = Architecture['NumOfHid_4']
			R_NeuralNet.NumOfHid_5 = Architecture['NumOfHid_5']
			R_NeuralNet.NumOfHid_6 = Architecture['NumOfHid_6']
			R_NeuralNet.NumOfHid_7 = Architecture['NumOfHid_7']
			R_NeuralNet.NumOfHid_8 = Architecture['NumOfHid_8']
			R_NeuralNet.NumOfHid_9 = Architecture['NumOfHid_9']
			R_NeuralNet.NumOfStates=Architecture['NumOfStates']
			R_NeuralNet.state = np.zeros((R_NeuralNet.NumOfStates,1))
			############################################
			R_NeuralNet.wieghts_1 = wieghts['wieghts_1']
			R_NeuralNet.wieghts_2 = wieghts['wieghts_2']
			R_NeuralNet.wieghts_3 = wieghts['wieghts_3']
			R_NeuralNet.wieghts_4 = wieghts['wieghts_4']
			R_NeuralNet.wieghts_5 = wieghts['wieghts_5']
			R_NeuralNet.wieghts_6 = wieghts['wieghts_6']
			R_NeuralNet.wieghts_7 = wieghts['wieghts_7']
			R_NeuralNet.wieghts_8 = wieghts['wieghts_8']
			R_NeuralNet.wieghts_9 = wieghts['wieghts_9']
			R_NeuralNet.wieghts_10 = wieghts['wieghts_10']
			R_NeuralNet.wieghts_11 = wieghts['wieghts_11']
			R_NeuralNet.wieghts_12 = wieghts['wieghts_12']
			R_NeuralNet.wieghts_13 = wieghts['wieghts_13']
			R_NeuralNet.wieghts_14 = wieghts['wieghts_14']
			R_NeuralNet.wieghts_15 = wieghts['wieghts_15']
			R_NeuralNet.wieghts_16 = wieghts['wieghts_16']
			R_NeuralNet.wieghts_17 = wieghts['wieghts_17']
			R_NeuralNet.wieghts_18 = wieghts['wieghts_18']
			R_NeuralNet.wieghts_19 = wieghts['wieghts_19']
			R_NeuralNet.wieghts_20 = wieghts['wieghts_20']
			R_NeuralNet.wieghts_21 = wieghts['wieghts_21']
			R_NeuralNet.wieghts_22 = wieghts['wieghts_22']
			R_NeuralNet.wieghts_23 = wieghts['wieghts_23']
			R_NeuralNet.wieghts_24 = wieghts['wieghts_24']
			R_NeuralNet.wieghts_25 = wieghts['wieghts_25']
			R_NeuralNet.wieghts_26 = wieghts['wieghts_26']
			R_NeuralNet.wieghts_27 = wieghts['wieghts_27']
			R_NeuralNet.wieghts_28 = wieghts['wieghts_28']
			R_NeuralNet.wieghts_29 = wieghts['wieghts_29']
			R_NeuralNet.wieghts_30 = wieghts['wieghts_30']
			R_NeuralNet.wieghts_31 = wieghts['wieghts_31']
			R_NeuralNet.wieghts_32 = wieghts['wieghts_32']
			R_NeuralNet.wieghts_33 = wieghts['wieghts_33']
			R_NeuralNet.wieghts_34 = wieghts['wieghts_34']
			R_NeuralNet.wieghts_35 = wieghts['wieghts_35']
			R_NeuralNet.wieghts_36 = wieghts['wieghts_36']
			R_NeuralNet.wieghts_37 = wieghts['wieghts_37']
			R_NeuralNet.wieghts_38 = wieghts['wieghts_38']
			R_NeuralNet.wieghts_39 = wieghts['wieghts_39']
			R_NeuralNet.wieghts_40 = wieghts['wieghts_40']
			R_NeuralNet.wieghts_41 = wieghts['wieghts_41']
			R_NeuralNet.wieghts_42 = wieghts['wieghts_42']
			R_NeuralNet.wieghts_43 = wieghts['wieghts_43']
			R_NeuralNet.wieghts_44 = wieghts['wieghts_44']
			R_NeuralNet.wieghts_45 = wieghts['wieghts_45']
			R_NeuralNet.wieghts_46 = wieghts['wieghts_46']
			R_NeuralNet.wieghts_47 = wieghts['wieghts_47']
			R_NeuralNet.wieghts_48 = wieghts['wieghts_48']
			R_NeuralNet.wieghts_49 = wieghts['wieghts_49']
			R_NeuralNet.wieghts_50 = wieghts['wieghts_50']
			R_NeuralNet.wieghts_51 = wieghts['wieghts_51']
			R_NeuralNet.wieghts_52 = wieghts['wieghts_52']
			R_NeuralNet.wieghts_53 = wieghts['wieghts_53']
			R_NeuralNet.wieghts_54 = wieghts['wieghts_54']
			R_NeuralNet.wieghts_55 = wieghts['wieghts_55']
			R_NeuralNet.wieghts_s1 = wieghts['wieghts_s1']
			R_NeuralNet.wieghts_s2 = wieghts['wieghts_s2']
			R_NeuralNet.wieghts_s3 = wieghts['wieghts_s3']
			R_NeuralNet.wieghts_s4 = wieghts['wieghts_s4']
			R_NeuralNet.wieghts_s5 = wieghts['wieghts_s5']
			R_NeuralNet.wieghts_s6 = wieghts['wieghts_s6']
			R_NeuralNet.wieghts_s7 = wieghts['wieghts_s7']
			R_NeuralNet.wieghts_s8 = wieghts['wieghts_s8']
			R_NeuralNet.wieghts_s9 = wieghts['wieghts_s9']
			R_NeuralNet.wieghts_s10 = wieghts['wieghts_s10']
			R_NeuralNet.wieghts_s11 = wieghts['wieghts_s11']
			R_NeuralNet.wieghts_s12 = wieghts['wieghts_s12']
			R_NeuralNet.wieghts_s13 = wieghts['wieghts_s13']
			R_NeuralNet.wieghts_s14 = wieghts['wieghts_s14']
			R_NeuralNet.wieghts_s15 = wieghts['wieghts_s15']
			R_NeuralNet.wieghts_s16 = wieghts['wieghts_s16']
			R_NeuralNet.wieghts_s17 = wieghts['wieghts_s17']
			R_NeuralNet.wieghts_s18 = wieghts['wieghts_s18']
			R_NeuralNet.wieghts_s19 = wieghts['wieghts_s19']
			R_NeuralNet.wieghts_s20 = wieghts['wieghts_s20']
			R_NeuralNet.BaisIn = wieghts['BaisIn']
			R_NeuralNet.BaisOut = wieghts['BaisOut']
			R_NeuralNet.BaisHid_1 = wieghts['BaisHid_1']
			R_NeuralNet.BaisHid_2 = wieghts['BaisHid_2']
			R_NeuralNet.BaisHid_3 = wieghts['BaisHid_3']
			R_NeuralNet.BaisHid_4 = wieghts['BaisHid_4']
			R_NeuralNet.BaisHid_5 = wieghts['BaisHid_5']
			R_NeuralNet.BaisHid_6 = wieghts['BaisHid_6']
			R_NeuralNet.BaisHid_7 = wieghts['BaisHid_7']
			R_NeuralNet.BaisHid_8 = wieghts['BaisHid_8']
			R_NeuralNet.BaisHid_9 = wieghts['BaisHid_9']
	except:
			R_NeuralNet = R_NeuralNet()
			print("warning the required files are ether incompatible or unavalable !, random creature was loaded.")
	return()

# fitness fuction
elementry for every genetic algorithm 


In [ ]:
@numba.njit()
def fitFinder(refrence , output , N , W):
	global ST_DIV 
	# two fitness methodes are used in parallel
	x = np.transpose(output)
	c = np.multiply((refrence -x),(refrence - x))
	sup_fit = W/(N*(c + 1))

	fit1 = np.sum(sup_fit)

	#scaled normal distribution
	x = ((refrence - x)**2)/(2*ST_DIV)
	sup_acc = np.exp(-x)/N
	fit2 = np.sum(W*sup_acc)


	return (fit1*fit2)/(fit1+fit2)


# accuracy function

In [ ]:
@numba.njit()
def accuracyFinder(refrence , output, N ,W):
	if (refrence.argmax() == output.argmax()) :

		return 1/N

	else :

		return 0


# data loader 
basically loads data to RAM

In [ ]:
def loadData(URL = os.path.join(path,"trainingset1.npy")):
	global trainingInput
	global trainingOutput
	data = np.load(URL,allow_pickle=True).item()
	l = len(data['result'])
	temp1 = np.zeros(((l//2,OUTPUT)))
	temp2 = np.zeros(((l//2,INPUT)))
	for i in range(l):
		if (i+1)%2 == 0 :
			temp1[i//2] = data['result'][i]
		else:
			temp2[i//2] = data['result'][i]
	trainingOutput = temp1
	trainingInput = temp2
	return ([trainingInput,trainingOutput])
def selectionMemory(R_NeuralNet , GEN):
	#store the fitness and accuracy of the best creature and then do the neccery calculations 
	global fitness_mem
	global accuracy_mem
	global SELECTION_RULE
	global MUTATION_RATE
	global MUT_ACC
	global RAND_PER_GEN
	fitness_mem.pop(0)
	accuracy_mem.pop(0)
	fitness_mem.append(float(R_NeuralNet.fitness))
	accuracy_mem.append(float(R_NeuralNet.accuracy))
	fit_sd = statistics.stdev(fitness_mem)
	acc_sd = statistics.stdev(accuracy_mem)
	fit_mean = statistics.mean(fitness_mem)
	acc_mean = statistics.mean(fitness_mem)
	MUT_ACC = (MUT_ACC_MAX)/(1 + (GEN/25)**2) 
	if RAND_PER_GEN > 0 :
		RAND_PER_GEN = RAND_PER_GEN - 1
	threashold = 0.0001
	mutationRateStep = 0.0005
	if((fit_sd < threashold*fit_mean) and (acc_sd < threashold*acc_mean)):
		if ( MUTATION_RATE > 0.01):
			MUTATION_RATE = MUTATION_RATE - mutationRateStep
		else : 
			MUTATION_RATE = 0.01
		print("selection rule : accuracy at :" + str(MUTATION_RATE)+ " mutation rate, mut acc :"+str(MUT_ACC))
		SELECTION_RULE = 0
	else:
		if ( MUTATION_RATE < 1 ):
			MUTATION_RATE = MUTATION_RATE + mutationRateStep
		print("selection rule : fitness // accuracy at :" + str(MUTATION_RATE)+ " mutation rate, mut acc :"+str(MUT_ACC))
		SELECTION_RULE = 1
	return()

# selection control function
the purpose of this fuction is to manipulate some of the algorithm parameters and the selection policy, to resolve the cases of Infertility

In [ ]:
@numba.njit()
def selectionControl(R_NeuralNet):
	global SELECTION_RULE
	if (SELECTION_RULE == 0):
		return R_NeuralNet.accuracy
	if (SELECTION_RULE == 1):
		return ((R_NeuralNet.accuracy + 10e-9)*(R_NeuralNet.fitness+ 10e-9)/(R_NeuralNet.fitness + R_NeuralNet.accuracy+ 20e-9))


# simulation method
applies the data sequentially to the nn and assign fitness and accuracy values to each creaure.

In [ ]:
def RunSimulation(A):
	fitness = np.empty((0),dtype = np.float32) 
	accuracies =np.empty((0),dtype = np.float32)
	#print(A[0].propagate(training_input))
	l = len(trainingInput)//8
	W = np.array([0.1667,0.1667,0.1667,0.1667,0.1667,0.1667]) #wieghtes of each output
	for i in tqdm(range(len(A))):
		aux_1 = 0.0
		aux_2 = 0.0
		for j in range(l):
			training_input = trainingInput[j]
			training_output = trainingOutput[j]
			try:
				out = np.array(A[i].propagate(training_input))
			except:
				out = np.array([10e8]*OUTPUT)
				out = np.reshape(out,(OUTPUT))
			aux_1 = aux_1 + fitFinder(training_output,out,l,W)
			aux_2 = aux_2 + accuracyFinder(training_output, out,l,W)

		A[i].fitness = aux_1
		A[i].accuracy = aux_2 
	return ()

#create the new populaion.
based on the values of accuracy and fintess and the currrent selection policy new population is created after each generaion.

In [ ]:
def CreateNewPopulation(A, GEN):
	#print(A[1].fitness)
	new_population = np.array(heapq.nlargest(BEST_FIT,A,key = lambda x: selectionControl( x ) ),dtype = R_NeuralNet )
	#extract the best creatures in the populatio
	#list of the new population 
	save(new_population[0])
	for i in range(len(new_population)):
		new_population[i].reset()
	# add some new random creatures to the population 
	for i in range(RAND_PER_GEN):
		new_population = np.append(new_population,R_NeuralNet())
	#reset the state in the new population to the initial state
	number_of_mutated_creatures = (SIZE_OF_POPULATION - BEST_FIT - RAND_PER_GEN)//RATIO_OF_MUT
	for i in range(number_of_mutated_creatures):
		new_population =  np.append(new_population,mutate(new_population[i%BEST_FIT].copy())) 
	#add mutated creatures to the population
	number_of_crossover_creatres = SIZE_OF_POPULATION - BEST_FIT - number_of_mutated_creatures - RAND_PER_GEN
	#the number of crossover creatures in the population
	for i in range(number_of_crossover_creatres):
		index = (random.randint(i + 1,BEST_FIT+i-1))%BEST_FIT
		#to avoid crossover of the creature with its self 
		new_population = np.append(new_population,new_population[i%BEST_FIT].crossOver(new_population[index]))
		#perform crossover
	#new population is ready 
	selectionMemory(new_population[0],GEN )
	return new_population
	#return the new population 

now RUN function will just go through the sequance of steps mentioned previously

In [ ]:
def RUN( NumOfGen = MAX_NUM_OF_GEN , Population = np.array([ R_NeuralNet() for j in range(SIZE_OF_POPULATION - NUM_OF_Z_C)] + [R_NeuralNet().zeroCreature() for x in range(NUM_OF_Z_C)]) ):
	l = len(Population)
	if (l < SIZE_OF_POPULATION):
	 	Population = np.append(Population,[ R_NeuralNet() for j in range(SIZE_OF_POPULATION - l)])
	# create a default random population for the firs time 
	for i in range(NumOfGen) :
		RunSimulation(Population)
		# runing the simulation

		print()
		print("##############################################")
		Population = CreateNewPopulation(Population,i)
		print("GEN : "+str(i)+" best fit : "+str(Population[0].fitness) +" / accuracy :" + str(Population[0].accuracy))
		Population[0].showStructure()
		print()
		fitt_plot.append(Population[0].fitness)
		acc_plot.append(Population[0].accuracy)
		plt.plot(list(range(len(fitt_plot))) , fitt_plot , 'r')
		plt.plot(list(range(len(acc_plot))) , acc_plot , 'b')
		plt.show()
		plt.pause(0.25)
		np.save(os.path.join(path, 'status.npy'),{'fit': fitt_plot , 'acc' : acc_plot})
		save(Population[0])

	return (Population)


# merge funtion

merges two populations together into one populaion through crossover

In [ ]:
def merge(PopulationA , PopulationB):
	#take the best creatures and then fell the rest with crossovers
	PopulationC = np.array([])
	#the best fit from each population 
	for i in range(SIZE_OF_POPULATION):
		PopulationC = np.append(PopulationC,(PopulationA[0].crossOver(PopulationB[0])).mutate())
	return PopulationC


# The tree method
please read the thises for more understanding

In [ ]:
def Tree(degree , brancheLenght = 20):
	if degree == 0 :
		return RUN(NumOfGen = brancheLenght)
	if degree > 0 :
		return RUN(NumOfGen = brancheLenght , Population = merge(Tree(degree - 1, brancheLenght),Tree(degree - 1 , brancheLenght)))


simple testing funcion

In [ ]:
def test(R_NeuralNet):
	fit,acc = RunSimulation([R_NeuralNet])
	print("fitness : " + str(fit) + "accuracy : " + str(acc))
	R_NeuralNet.showStructure()
	return()

In [ ]:

loadData()

start = time.time()
#x = R_NeuralNet()
#load(x)
#y = R_NeuralNet()
#load(y)
#status = np.load(os.path.join(path, 'status.npy'),allow_pickle=True).item()
fitt_plot = status['fit'] 
acc_plot = status['acc']
#results = RUN(NumOfGen = 100,Population = [y])
results = RUN(NumOfGen = 100)
end = time.time()
print(end - start)
save(results[0])